# Advanced Marketing Mix Modeling (MMM) Pipeline
## Global B2B Multi-Regional, Multi-Product Implementation

### What This Notebook Does

**Marketing Mix Modeling (MMM)** answers: *"For every dollar we spend on LinkedIn vs. Google vs. Display, how much revenue do we get back?"*

Unlike digital attribution (last-click), MMM uses **statistical regression** to estimate causal effects of media spend on revenue, accounting for:
- **Time delays** (spend today → revenue in 6-9 months for B2B)
- **Diminishing returns** (doubling spend doesn't double results)
- **External factors** (seasonality, economic conditions)

### Core Techniques (cells 4-7)

| Technique | What It Does | Why We Need It |
|-----------|--------------|----------------|
| **Geometric Adstock** | Spreads each week's spend across future weeks with exponential decay | LinkedIn ads today still influence buyers 4 weeks from now |
| **Hill Saturation** | S-curve transformation that flattens at high spend | The 10th impression to the same person has ~0 value |
| **Nevergrad Optimization** | Evolutionary algorithm to find best decay/saturation params | Can't grid-search 60 parameters; no gradients available |
| **Ridge Regression** | L2-penalized linear model with positive constraints | Prevents overfitting; ensures "more spend ≠ less revenue" |

### Validation & Uncertainty (cells 6, 8-9)

| Feature | Purpose |
|---------|---------|
| **Time-Series CV** | Tests if model can predict FUTURE quarters (not just fit history) |
| **Bootstrap CI** | Shows "LinkedIn ROI is 3.2x [2.8-3.6]" not just "3.2x" |
| **MAPE Metric** | "We're typically 12% off" is more intuitive than R² |

### Output (cells 12-13)

Results are saved to `ATOMIC.MMM_MODEL_RESULT` with ROI, confidence intervals, marginal ROI, and recommended spend per Channel × Region × Product.

---
**Data Sources**
- **Input**: `DIMENSIONAL.V_MMM_INPUT_WEEKLY` (weekly spend + revenue + controls)
- **Output**: `ATOMIC.MMM_MODEL_RESULT` (joins to dimension tables via FK)

**Infrastructure**: Snowflake Notebooks or SPCS | Python 3.11 | ~5-10 min runtime


In [ ]:
# =============================================================================
# CELL 0: Install Required Packages
# =============================================================================
# Note: nevergrad and snowflake-ml-python are not pre-installed in Snowflake notebooks
# This cell requires EXTERNAL_ACCESS_INTEGRATIONS to be configured
# Using os.system() for compatibility with headless SPCS execution

import os
import sys

packages = ["nevergrad", "snowflake-ml-python"]
for pkg in packages:
    print(f"Installing {pkg}...")
    result = os.system(f"{sys.executable} -m pip install {pkg} -q")
    print(f"  {'✓ Done' if result == 0 else '⚠ Failed'}")

In [ ]:
# =============================================================================
# CELL 1: Imports and Configuration
# =============================================================================
# 
# KEY LIBRARIES:
# - nevergrad: Meta's derivative-free optimization library. We use it because
#   MMM hyperparameters (adstock decay, saturation) don't have clean gradients.
#   TwoPointsDE is an evolutionary algorithm that works well for ~10-100 params.
#
# - Ridge regression: Linear model with L2 penalty. We use Ridge (not OLS) because:
#   (1) Marketing data is often collinear (channels spike together in Q4)
#   (2) L2 shrinks coefficients toward zero, preventing wild estimates
#   (3) We can't use Lasso (L1) because it zeros out channels entirely
#
# - scipy.optimize: For budget reallocation with business constraints (±30% limits)
#
# ─────────────────────────────────────────────────────────────────────────────
# LIBRARY CONTEXT FOR MAINTAINERS
# ─────────────────────────────────────────────────────────────────────────────
#
# pandas/numpy: Standard data manipulation. No special versions required.
#
# sklearn.linear_model.Ridge:
#   - Implements: β = (X'X + λI)⁻¹X'y (closed-form, fast)
#   - alpha parameter = λ in the penalty term
#   - We use default alpha=1.0 which is mild regularization
#
# sklearn.preprocessing.StandardScaler:
#   - Transforms X to zero mean, unit variance: X_scaled = (X - μ) / σ
#   - Critical for Ridge: penalty treats all features equally, so they must
#     be on the same scale. Without scaling, a $1M spend column would have
#     tiny coefficients vs. a [0,1] saturation column with huge coefficients.
#
# nevergrad:
#   - Meta's library for "black-box" optimization (no gradients needed)
#   - TwoPointsDE: Differential Evolution variant, population-based
#   - "budget" = number of function evaluations (not $$ budget!)
#   - See: https://facebookresearch.github.io/nevergrad/
#
# scipy.optimize.minimize:
#   - General-purpose minimizer with many algorithms
#   - method='SLSQP': Sequential Least Squares Programming
#     Handles equality + inequality constraints efficiently
#   - We use it for budget allocation where SLSQP's constraint handling shines
#
# ALTERNATIVES CONSIDERED (not used):
# - PyMC/Stan: Bayesian MMM (e.g., Robyn). More principled uncertainty but
#   10x slower and requires MCMC tuning expertise.
# - LightweightMMM (Google): Similar approach but JAX-based. We chose sklearn
#   for broader compatibility with Snowflake's Python environment.
# - CausalImpact: Good for single interventions, not continuous spend allocation.
# =============================================================================

import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
import json

# Snowflake
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F

# ML/Stats
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import nevergrad as ng
from scipy.optimize import minimize

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')
np.random.seed(42)  # Reproducibility for bootstrap sampling

# Configuration
@dataclass
class MMMConfig:
    """
    Configuration for MMM model training.
    
    GRANULARITY CHOICES:
    - geo_level: Controls geographic aggregation. Options:
        - "GLOBAL": Aggregate all regions (best for sparse data, ~10 channel groups)
        - "SUPER_REGION": 3-4 regions per channel (needs 50+ weeks per combo)
        - "REGION" or "COUNTRY": More granular (needs very rich data)
      Rule of thumb: need ~50+ weeks of non-zero REVENUE per Channel×Geo combo.
    
    - product_level: SEGMENT (4 groups) vs CATEGORY (23 groups). More granular = 
      more actionable but requires more data. Start with SEGMENT, drill down if R² holds.
    
    HYPERPARAMETER SEARCH:
    - nevergrad_budget: 500 iterations is a good balance. Robyn uses 2000+ but we're
      optimizing fewer params (no decomposition). Increase if CV MAPE is unstable.
    
    VALIDATION:
    - cv_train_weeks=52: Full year captures seasonality (Q1 budget flush, Q4 holidays)
    - cv_test_weeks=13: Quarter-out holdout mimics real forecasting use case
    
    DATA SPARSITY NOTE:
    If CV MAPE > 50%, the data is likely too sparse for the chosen granularity.
    Switch geo_level to "GLOBAL" to aggregate across regions and improve model stability.
    """
    # Data sources
    input_view: str = "DIMENSIONAL.V_MMM_INPUT_WEEKLY"
    output_table: str = "ATOMIC.MMM_MODEL_RESULT"
    
    # Model granularity - use GLOBAL for channel-only modeling (most robust)
    # Use SUPER_REGION only if you have 50+ weeks with revenue per channel-region
    geo_level: str = "GLOBAL"         # GLOBAL (recommended), SUPER_REGION, REGION, or COUNTRY
    product_level: str = "SEGMENT"    # SEGMENT, DIVISION, or CATEGORY
    
    # Hyperparameter optimization
    nevergrad_budget: int = 500       # Evolutionary algorithm iterations
    # ridge_alpha: float = 10.0       # DEMO: Lower regularization → wilder ROI estimates (try this to show overfitting)
    ridge_alpha: float = 50.0         # L2 penalty strength (stronger regularization → more conservative, realistic ROI)
    
    # Time-series cross-validation (rolling window, never peek at future)
    cv_train_weeks: int = 52          # 1 year training window
    cv_test_weeks: int = 13           # 1 quarter holdout (13 weeks)
    cv_step_weeks: int = 13           # Roll forward 1 quarter between folds
    
    # Bootstrap for uncertainty quantification
    n_bootstrap: int = 100            # Resample iterations (100 is standard)
    confidence_level: float = 0.90    # 90% CI = 5th to 95th percentile
    
    # Budget optimizer constraints
    budget_change_limit: float = 0.30 # ±30% per channel (realistic for CMO approval)
    
    # Model versioning (for tracking in MMM_MODEL_RESULT table and Model Registry)
    # Uses semantic prefix + timestamp for auto-increment: v3_1_YYYYMMDD_HHMMSS
    _version_prefix: str = "v3_1"
    model_version: str = field(default_factory=lambda: f"v3_1_{datetime.now().strftime('%Y%m%d_%H%M%S')}")

config = MMMConfig()
print(f"MMM Configuration initialized: {config.model_version}")


In [ ]:
# =============================================================================
# CELL 2: Connect to Snowflake and Load Data
# =============================================================================

session = get_active_session()
print(f"Connected to Snowflake: {session.get_current_database()}.{session.get_current_schema()}")

# Load weekly aggregated data from dimensional view
df_raw = session.table(config.input_view).to_pandas()

# Standardize column names to uppercase
df_raw.columns = df_raw.columns.str.upper()

# Map view column names to expected model column names
# The view uses _NAME/_CODE suffixes, but the model expects simple names
column_mapping = {
    'SUPER_REGION_NAME': 'SUPER_REGION',
    'REGION_NAME': 'REGION',
    'COUNTRY_NAME': 'COUNTRY',
    'SEGMENT_NAME': 'SEGMENT',
    'DIVISION_NAME': 'DIVISION',
    'CATEGORY_NAME': 'CATEGORY',
    'CHANNEL_CODE': 'CHANNEL',
    'AVG_PMI': 'PMI_INDEX',
    'AVG_COMPETITOR_SOV': 'COMPETITOR_SOV',
    'AVG_INDUSTRY_GROWTH': 'INDUSTRY_GROWTH'
}
df_raw = df_raw.rename(columns=column_mapping)

print(f"\nLoaded {len(df_raw):,} rows from {config.input_view}")
print(f"Date range: {df_raw['WEEK_START'].min()} to {df_raw['WEEK_START'].max()}")
print(f"\nColumns: {df_raw.columns.tolist()}")

# Check dimension coverage
print(f"\nDimension coverage:")
print(f"  SUPER_REGION: {df_raw['SUPER_REGION'].dropna().unique().tolist()}")
print(f"  CHANNEL: {df_raw['CHANNEL'].dropna().unique().tolist()}")
segment_vals = df_raw['SEGMENT'].dropna().unique().tolist() if 'SEGMENT' in df_raw.columns and df_raw['SEGMENT'].notna().any() else []
print(f"  SEGMENT: {segment_vals if segment_vals else 'All NULL - will use ALL'}")

# Preview data
df_raw.head()


In [ ]:
# =============================================================================
# CELL 3: Data Preparation and Feature Engineering
# =============================================================================
#
# WHY THESE FEATURES MATTER:
#
# 1. COMPOSITE KEYS (Channel_Region_Product):
#    We model each combination separately because "LinkedIn in EMEA for Safety"
#    behaves differently than "LinkedIn in APAC for Healthcare". This is the core
#    value prop: granular ROI, not just "LinkedIn overall".
#
# 2. FOURIER TERMS FOR SEASONALITY:
#    Instead of 52 dummy variables (one per week), we use sin/cos waves.
#    - SIN_1/COS_1: Annual cycle (captures "Q4 always high")
#    - SIN_2/COS_2: Semi-annual (captures "Q2 and Q4 different from Q1/Q3")
#    - SIN_3/COS_3: Quarterly patterns
#    Benefit: 6 features instead of 52, prevents overfitting, captures smooth patterns.
#
# 3. TREND COMPONENT:
#    Linear time trend captures "revenue grows 5% per year regardless of marketing".
#    Without this, model would attribute organic growth to whichever channel scaled up.
#
# 4. Q1/Q3 FLAGS:
#    B2B-specific: Q1 = budget flush, Q3 = pre-year-end push. Binary flags let the
#    model learn "Q1 has 20% higher baseline" without needing exact week-of-year.
#
# ─────────────────────────────────────────────────────────────────────────────
# DEEPER DIVE: FOURIER SEASONALITY (From Signal Processing)
# ─────────────────────────────────────────────────────────────────────────────
#
# Any periodic signal can be decomposed into a sum of sinusoids (Fourier theorem).
# For a signal with period T (52 weeks for annual seasonality):
#
#   f(t) = a₀ + Σₖ [aₖ·cos(2πkt/T) + bₖ·sin(2πkt/T)]
#
# Each k represents a "harmonic" of the fundamental frequency:
#   k=1: Fundamental (one complete cycle per year)
#   k=2: First harmonic (two cycles per year, i.e., semi-annual)
#   k=3: Second harmonic (four cycles, i.e., quarterly)
#
# WHY SIN AND COS PAIRS:
# A single sinusoid aₖ·cos(2πkt/T + φ) has both amplitude aₖ and phase φ.
# Using both sin AND cos with separate coefficients, the regression can learn
# any amplitude and phase:
#   aₖ·cos(2πkt/T + φ) = (aₖ·cos(φ))·cos(2πkt/T) + (aₖ·sin(φ))·sin(2πkt/T)
#                       = β_cos·cos(2πkt/T) + β_sin·sin(2πkt/T)
#
# So the model learns β_cos and β_sin, and we get:
#   Amplitude = √(β_cos² + β_sin²)
#   Phase = arctan(β_sin / β_cos)
#
# WHY 3 HARMONICS:
# - k=1,2,3 capture annual, semi-annual, and quarterly patterns
# - Higher harmonics (k=4,5,...) would capture weekly fluctuations
# - More harmonics = more flexible but higher overfitting risk
# - Rule of thumb: use k = 1 to (n/2-1) where n = periods per year ÷ 10
#   For weekly data with 52 periods/year: up to k ≈ 3 is reasonable
#
# ALTERNATIVE: SEASONAL DUMMIES
# 52 binary indicators (one per week) is equivalent to unlimited harmonics.
# Downsides: 52 extra parameters, can't extrapolate, captures noise not signal.
# =============================================================================

def prepare_mmm_data(df: pd.DataFrame, config: MMMConfig) -> pd.DataFrame:
    """
    Prepare data for MMM modeling with proper granularity and features.
    
    Features added:
    - Composite keys for channel × region × product (granular attribution)
    - Fourier seasonality (smooth annual/semi-annual patterns, 6 features vs 52 dummies)
    - Linear trend (isolate organic growth from marketing impact)
    - B2B fiscal flags (Q1 budget flush, Q3 push)
    """
    df = df.copy()
    
    # Ensure datetime
    df['WEEK_START'] = pd.to_datetime(df['WEEK_START'])
    
    # Fill missing values
    numeric_cols = ['SPEND', 'IMPRESSIONS', 'CLICKS', 'REVENUE', 'PMI_INDEX', 'COMPETITOR_SOV']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    
    # Create composite dimension keys based on config granularity
    geo_col = config.geo_level
    prod_col = config.product_level
    
    # Handle GLOBAL geo_level - aggregate all regions into single "GLOBAL" value
    # This is recommended for sparse data to ensure sufficient sample size per channel
    if geo_col == "GLOBAL":
        print(f"  Using GLOBAL geo aggregation (channel-only modeling)")
        df['GEO_KEY'] = 'GLOBAL'
        geo_col = 'GEO_KEY'
    else:
        print(f"  Looking for geo_col='{geo_col}' in columns: {'YES' if geo_col in df.columns else 'NO'}")
        if geo_col in df.columns:
            df[geo_col] = df[geo_col].fillna('UNKNOWN')
        else:
            df[geo_col] = 'ALL'
            print(f"  WARNING: {geo_col} column not found, using 'ALL'")
    
    # Debug: print what columns we have
    print(f"  Looking for prod_col='{prod_col}' in columns: {'YES' if prod_col in df.columns else 'NO'}")
    print(f"  Looking for 'CHANNEL' in columns: {'YES' if 'CHANNEL' in df.columns else 'NO'}")
        
    if prod_col in df.columns:
        df[prod_col] = df[prod_col].fillna('ALL')  # Use 'ALL' for null product since we don't have segment data
    else:
        df[prod_col] = 'ALL'
        print(f"  WARNING: {prod_col} column not found, using 'ALL'")
        
    if 'CHANNEL' in df.columns:
        df['CHANNEL'] = df['CHANNEL'].fillna('UNKNOWN')
    else:
        df['CHANNEL'] = 'UNKNOWN'
        print("  WARNING: CHANNEL column not found!")
    
    # Composite key: Channel_Region_Product
    # With GLOBAL geo_level, this becomes Channel_GLOBAL_ALL (effectively channel-only)
    df['CHANNEL_KEY'] = (
        df['CHANNEL'].astype(str) + '_' + 
        df[geo_col].astype(str) + '_' + 
        df[prod_col].astype(str)
    )
    
    # Add time features for seasonality
    df['WEEK_OF_YEAR'] = df['WEEK_START'].dt.isocalendar().week.astype(int)
    df['YEAR'] = df['WEEK_START'].dt.year
    df['TREND'] = (df['WEEK_START'] - df['WEEK_START'].min()).dt.days / 365.25
    
    # Fourier terms for seasonality (annual cycle)
    for k in [1, 2, 3]:
        df[f'SIN_{k}'] = np.sin(2 * np.pi * k * df['WEEK_OF_YEAR'] / 52)
        df[f'COS_{k}'] = np.cos(2 * np.pi * k * df['WEEK_OF_YEAR'] / 52)
    
    # Q1/Q3 seasonality flag (B2B budget cycles)
    df['Q1_FLAG'] = ((df['WEEK_START'].dt.month >= 1) & (df['WEEK_START'].dt.month <= 3)).astype(int)
    df['Q3_FLAG'] = ((df['WEEK_START'].dt.month >= 7) & (df['WEEK_START'].dt.month <= 9)).astype(int)
    
    return df

# Prepare data
df = prepare_mmm_data(df_raw, config)

# Summary statistics
print(f"\nUnique channel keys: {df['CHANNEL_KEY'].nunique()}")
print(f"Unique {config.geo_level}: {df[config.geo_level].nunique() if config.geo_level in df.columns else 'N/A'}")
print(f"Unique {config.product_level}: {df[config.product_level].nunique() if config.product_level in df.columns else 'N/A'}")
print(f"\nSample channel keys: {df['CHANNEL_KEY'].unique()[:10]}")


In [ ]:
# =============================================================================
# CELL 4: Adstock and Saturation Transformation Functions
# =============================================================================
#
# THESE ARE THE TWO CORE CONCEPTS IN MODERN MMM:
#
# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ 1. ADSTOCK (Carryover Effect)                                               │
# │                                                                             │
# │ PROBLEM: A $100k LinkedIn campaign on Week 1 doesn't just affect Week 1.   │
# │ In B2B, someone sees an ad, researches for 3 weeks, then converts.         │
# │                                                                             │
# │ SOLUTION: "Spread" the spend across future weeks with exponential decay:   │
# │                                                                             │
# │   Week 1: $100k    →  Effective: $100k                                     │
# │   Week 2: $0       →  Effective: $70k  (70% of previous)                   │
# │   Week 3: $0       →  Effective: $49k  (70% of $70k)                       │
# │   Week 4: $0       →  Effective: $34k  ...and so on                        │
# │                                                                             │
# │ THETA PARAMETER:                                                            │
# │   - theta = 0.0: No carryover (Search ads, immediate intent)               │
# │   - theta = 0.5: Medium carryover (Facebook, consideration)                │
# │   - theta = 0.8: Long carryover (LinkedIn B2B, 6-8 week cycles)            │
# │   - theta = 0.95: Very long (TV brand campaigns, months of effect)         │
# │                                                                             │
# │ WHY GEOMETRIC: Simple (1 param), interpretable, matches empirical data.    │
# │ Alternative: Weibull allows delayed peak (effect maxes at week 3).         │
# └─────────────────────────────────────────────────────────────────────────────┘
#
# MATHEMATICAL INTUITION FOR ADSTOCK:
#
# The geometric adstock is a first-order autoregressive (AR(1)) filter:
#   x_eff[t] = x[t] + θ·x_eff[t-1]
#
# Expanding recursively:
#   x_eff[t] = x[t] + θ·x[t-1] + θ²·x[t-2] + θ³·x[t-3] + ...
#
# This is an infinite geometric series. For constant spend s, the steady-state is:
#   x_eff_∞ = s·(1 + θ + θ² + ...) = s/(1-θ)
#
# Half-life interpretation: Effect decays to 50% after ln(0.5)/ln(θ) periods.
#   θ=0.7 → half-life ≈ 1.9 weeks (effect halves in ~2 weeks)
#   θ=0.9 → half-life ≈ 6.6 weeks (effect persists much longer)
#
# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ 2. SATURATION / HILL FUNCTION (Diminishing Returns)                         │
# │                                                                             │
# │ PROBLEM: Doubling spend doesn't double revenue. At some point, you've      │
# │ reached everyone interested. The 10th impression to the same person        │
# │ has near-zero incremental value.                                           │
# │                                                                             │
# │ SOLUTION: S-curve transformation (Hill function from pharmacology):        │
# │                                                                             │
# │   Response │              ●●●●●●●●●●●●  ← Saturation (flat)               │
# │      ▲     │         ●●●●●                                                 │
# │      │     │      ●●●                    ← Efficient zone                  │
# │      │     │    ●●                                                         │
# │      │     │  ●●                                                           │
# │      └─────┴──●───────────────────────► Spend                              │
# │              gamma (half-saturation point)                                 │
# │                                                                             │
# │ PARAMETERS:                                                                 │
# │   - alpha (shape): How steep the S-curve is. Higher = sharper transition.  │
# │   - gamma (scale): Spend level where response = 50% of max.                │
# │                    If gamma = $50k, then at $50k spend you're at 50%.      │
# │                                                                             │
# │ WHY HILL: Bounded [0,1], interpretable gamma, used by Robyn/LightweightMMM │
# └─────────────────────────────────────────────────────────────────────────────┘
#
# MATHEMATICAL INTUITION FOR HILL SATURATION:
#
# The Hill function: f(x) = x^α / (x^α + γ^α)
#
# Key properties (useful for understanding marginal returns):
#   - f(0) = 0, f(∞) → 1  (bounded response, asymptotes at max)
#   - f(γ) = 0.5 exactly  (γ is the "half-maximal effective dose" or EC50)
#   - Derivative: f'(x) = α·γ^α·x^(α-1) / (x^α + γ^α)²
#     → Marginal response DECREASES as x increases (diminishing returns)
#     → At x = γ, marginal response is α/(4γ)
#
# Alpha controls the "steepness" of the S-curve:
#   - α < 1: Concave throughout (rapid early saturation, like commodity goods)
#   - α = 1: Standard rectangular hyperbola f(x) = x/(x+γ) (Michaelis-Menten)
#   - α > 1: Sigmoid with inflection point (threshold effect, then saturation)
#            Inflection at x = γ·((α-1)/(α+1))^(1/α)
#
# The Hill function originates from enzyme kinetics (Michaelis-Menten) and
# pharmacology (Hill coefficient for cooperative binding). In marketing, it
# models audience saturation: eventually everyone who will respond, has.
# =============================================================================

def geometric_adstock(x: np.ndarray, theta: float) -> np.ndarray:
    """
    Geometric Adstock Transformation (Carryover Effect).
    
    Models the "memory" of advertising: this week's effective spend includes
    decayed contributions from all prior weeks. Equivalent to an infinite
    geometric series: x_eff[t] = x[t] + θ*x[t-1] + θ²*x[t-2] + ...
    
    Formula: x_adstocked[t] = x[t] + theta * x_adstocked[t-1]
    
    Parameters:
    -----------
    x : array - Raw spend values (weekly)
    theta : float - Decay rate (0 to 1). The "half-life" is ln(0.5)/ln(θ) weeks.
                   - LinkedIn B2B: 0.7-0.9 (long consideration cycle)
                   - Paid Search: 0.1-0.3 (immediate intent, fast decay)
                   - Display: 0.4-0.6 (awareness, medium decay)
    
    Returns:
    --------
    x_adstocked : array - Transformed values reflecting cumulative exposure
    """
    x = np.asarray(x, dtype=float)
    x_adstocked = np.zeros_like(x)
    
    if len(x) == 0:
        return x_adstocked
    
    # Recursive computation: each period inherits θ fraction of previous
    x_adstocked[0] = x[0]
    for t in range(1, len(x)):
        x_adstocked[t] = x[t] + theta * x_adstocked[t-1]
    
    return x_adstocked


def hill_saturation(x: np.ndarray, alpha: float, gamma: float) -> np.ndarray:
    """
    Hill Function for Saturation (Diminishing Returns).
    
    Maps spend to a 0-1 scale representing "response intensity". At gamma spend,
    response is exactly 0.5 (50% of maximum possible). This is the "half-EC50"
    concept from pharmacology applied to marketing.
    
    Formula: x^α / (x^α + γ^α)
    
    Parameters:
    -----------
    x : array - Adstocked spend values (apply adstock FIRST, then saturation)
    alpha : float - Shape/slope parameter (typically 0.5 to 3.0)
                   - alpha < 1: Concave from origin (quick saturation)
                   - alpha = 1: Standard hyperbolic
                   - alpha > 1: S-curve with inflection point (slow start, then steep)
    gamma : float - Half-saturation point. Spend level where response = 50% of max.
                   Typically set relative to observed spend range (e.g., median spend).
    
    Returns:
    --------
    x_saturated : array - Values in [0, 1] representing response intensity
    
    Note: Final revenue contribution = coefficient × saturated_value
    """
    x = np.asarray(x, dtype=float)
    x = np.maximum(x, 0)  # No negative spend
    gamma = max(gamma, 1e-10)  # Avoid division by zero
    
    # Hill function: asymptotes to 1 as x → ∞
    x_saturated = (x ** alpha) / (x ** alpha + gamma ** alpha)
    return x_saturated


def apply_media_transformations(
    X: pd.DataFrame, 
    params: Dict[str, Dict[str, float]], 
    channels: List[str]
) -> pd.DataFrame:
    """Apply adstock and saturation transformations to all media channels."""
    X_transformed = X.copy()
    
    for ch in channels:
        if ch not in X.columns or ch not in params:
            continue
            
        p = params[ch]
        x_raw = X[ch].values
        
        # Step 1: Adstock (carryover)
        x_adstocked = geometric_adstock(x_raw, p['theta'])
        
        # Step 2: Saturation (diminishing returns)
        x_saturated = hill_saturation(x_adstocked, p['alpha'], p['gamma'])
        
        X_transformed[ch] = x_saturated
    
    return X_transformed

# Demonstrate transformations
print("Transformation functions defined.")
print("\nExample: Geometric adstock with theta=0.7")
sample_spend = np.array([100, 0, 0, 0, 0, 50, 0, 0])
sample_adstock = geometric_adstock(sample_spend, theta=0.7)
print(f"Raw spend:     {sample_spend}")
print(f"Adstocked:     {np.round(sample_adstock, 1)}")


In [ ]:
# =============================================================================
# CELL 5: Pivot Data to Wide Format for Modeling
# =============================================================================
#
# DATA SHAPE TRANSFORMATION:
#
# Input (long format):
#   WEEK  |  CHANNEL_KEY           | SPEND  | REVENUE
#   W1    |  LINKEDIN_EMEA_SI      | 50000  | 100000
#   W1    |  GOOGLE_EMEA_SI        | 30000  | 100000
#   W1    |  LINKEDIN_APAC_HC      | 20000  | 100000
#   W2    |  LINKEDIN_EMEA_SI      | 45000  | 105000
#   ...
#
# Output (wide format for regression):
#   WEEK | LINKEDIN_EMEA_SI | GOOGLE_EMEA_SI | LINKEDIN_APAC_HC | ... | REVENUE
#   W1   | 50000            | 30000          | 20000            | ... | 100000
#   W2   | 45000            | 25000          | 22000            | ... | 105000
#
# WHY WIDE FORMAT:
# Regression needs y ~ X1 + X2 + X3 + ...
# Each column is a "feature" (channel×region×product combination)
# Each row is an observation (week)
#
# MIN_SPEND_THRESHOLD:
# Channels with < $1000 total spend are dropped because:
#   - Not enough signal to estimate effect reliably
#   - Adds noise and parameters without benefit
#   - Can cause numerical instability in optimization
# =============================================================================

def pivot_for_modeling(
    df: pd.DataFrame, 
    config: MMMConfig,
    min_spend_threshold: float = 1000
) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, List[str]]:
    """
    Pivot data to wide format for regression modeling.
    
    Transforms long-format data (one row per week×channel) to wide format
    (one row per week, one column per channel). Filters out low-spend channels.
    
    Returns:
    --------
    X_media : DataFrame - Media spend variables (to be adstock/saturation transformed)
    y : Series - Target variable (total revenue per week)
    X_control : DataFrame - Control variables (seasonality, PMI, SOV)
    channels : List - Channel keys with sufficient data for modeling
    """
    # Aggregate by week and channel_key
    df_agg = df.groupby(['WEEK_START', 'CHANNEL_KEY']).agg({
        'SPEND': 'sum',
        'IMPRESSIONS': 'sum',
        'CLICKS': 'sum',
        'REVENUE': 'sum'
    }).reset_index()
    
    # Pivot spend to wide format
    X_media = df_agg.pivot_table(
        index='WEEK_START',
        columns='CHANNEL_KEY',
        values='SPEND',
        aggfunc='sum'
    ).fillna(0).sort_index()
    
    # Filter channels with minimum spend
    channel_totals = X_media.sum()
    valid_channels = channel_totals[channel_totals >= min_spend_threshold].index.tolist()
    X_media = X_media[valid_channels]
    
    # Target: Total revenue per week
    y = df.groupby('WEEK_START')['REVENUE'].sum().sort_index()
    
    # Control variables
    control_cols = ['TREND', 'SIN_1', 'COS_1', 'SIN_2', 'COS_2', 'Q1_FLAG', 'Q3_FLAG']
    if 'PMI_INDEX' in df.columns:
        control_cols.append('PMI_INDEX')
    if 'COMPETITOR_SOV' in df.columns:
        control_cols.append('COMPETITOR_SOV')
    
    X_control = df.groupby('WEEK_START')[control_cols].first().sort_index()
    
    # Align indices
    common_idx = X_media.index.intersection(y.index).intersection(X_control.index)
    X_media = X_media.loc[common_idx]
    y = y.loc[common_idx]
    X_control = X_control.loc[common_idx]
    
    channels = X_media.columns.tolist()
    
    return X_media, y, X_control, channels

# Pivot data
X_media, y, X_control, channels = pivot_for_modeling(df, config)

print(f"\nModeling {len(channels)} channel-region-product combinations")
print(f"Time periods: {len(y)} weeks")
print(f"Total spend: ${X_media.sum().sum():,.0f}")
print(f"Total revenue: ${y.sum():,.0f}")
print(f"\nControl variables: {X_control.columns.tolist()}")
print(f"\nTop 10 channels by spend:")
print(X_media.sum().sort_values(ascending=False).head(10))


In [ ]:
# =============================================================================
# CELL 6: Time-Series Cross-Validation
# =============================================================================
#
# WHY TIME-SERIES CV INSTEAD OF K-FOLD:
#
# Standard k-fold CV randomly shuffles data, which would let us "peek" at future
# weeks when predicting past weeks. This causes overly optimistic metrics because
# the model learns patterns it wouldn't have access to in production.
#
# Time-series CV respects temporal order:
#
#   Fold 1: Train [Week 1-52]  → Test [Week 53-65]   (predict Q1 2024)
#   Fold 2: Train [Week 14-65] → Test [Week 66-78]  (predict Q2 2024)
#   Fold 3: Train [Week 27-78] → Test [Week 79-91]  (predict Q3 2024)
#   ...
#
# This mimics real use: "Given everything up to today, how well can we predict
# next quarter?" If CV MAPE is 12%, expect 12% error in actual forecasts.
#
# METRIC CHOICES:
# - MAPE (Mean Absolute Percentage Error): "On average, we're off by X%"
#   Industry standard for MMM. Target: <15% is good, <10% is excellent.
# - NRMSE: RMSE normalized by mean. Comparable across different revenue scales.
# - R²: Variance explained. >0.85 for in-sample, >0.70 for CV is solid.
#
# ─────────────────────────────────────────────────────────────────────────────
# DEEPER DIVE: UNDERSTANDING EACH METRIC
# ─────────────────────────────────────────────────────────────────────────────
#
# R² (Coefficient of Determination):
#   R² = 1 - SS_res / SS_tot = 1 - Σ(y - ŷ)² / Σ(y - ȳ)²
#
#   Interpretation: "What fraction of the variance in y does our model explain?"
#   - R² = 1.0: Perfect predictions (ŷ = y for all points)
#   - R² = 0.0: Model predicts the mean every time (useless)
#   - R² < 0:   Model is WORSE than predicting the mean (possible in CV!)
#
#   Caution: R² can be artificially high if y has a strong trend. A model that
#   just predicts "revenue goes up 5% per year" might get R² = 0.8 without
#   capturing any marketing effects. That's why we include TREND as a control.
#
# RMSE (Root Mean Squared Error):
#   RMSE = √[Σ(y - ŷ)² / n]
#
#   Same units as y (dollars). Penalizes large errors heavily due to squaring.
#   Good for: "On average, how many dollars off are we?"
#   Weakness: Not comparable across different revenue scales.
#
# MAE (Mean Absolute Error):
#   MAE = Σ|y - ŷ| / n
#
#   More robust to outliers than RMSE (no squaring). Also in dollars.
#   If MAE << RMSE, you have some big outliers (worth investigating).
#
# MAPE (Mean Absolute Percentage Error):
#   MAPE = (1/n) · Σ|y - ŷ| / |y| × 100
#
#   The "headline" metric for business stakeholders.
#   - Scale-free (%) so comparable across regions/products
#   - Intuitive: "We're typically 12% off"
#   - Weakness: Undefined when y = 0 (we mask those out)
#   - Weakness: Asymmetric—50% under-prediction feels same as 100% over-prediction
#
# NRMSE (Normalized RMSE):
#   NRMSE = RMSE / mean(y) × 100
#
#   Percentage scale like MAPE, but uses RMSE instead of MAE.
#   Useful for comparing model quality across different datasets.
#
# WHY WE REPORT MULTIPLE METRICS:
# No single metric tells the whole story. R² shows explanatory power, MAPE
# shows practical accuracy, RMSE reveals if large errors exist. Together they
# give a complete picture of model quality.
# =============================================================================

def time_series_cv_split(
    n_samples: int,
    train_size: int,
    test_size: int,
    step_size: int
) -> List[Tuple[np.ndarray, np.ndarray]]:
    """
    Generate time-series cross-validation splits (rolling window).
    
    Unlike k-fold, this NEVER lets the model see future data during training.
    Each fold trains on [t, t+train_size) and tests on [t+train_size, t+train_size+test_size).
    
    With train=52, test=13, step=13:
    - ~4 folds per 2 years of data
    - Each fold predicts a full quarter ahead
    - Realistic for "next quarter budget planning" use case
    """
    splits = []
    start = 0
    while start + train_size + test_size <= n_samples:
        train_idx = np.arange(start, start + train_size)
        test_idx = np.arange(start + train_size, start + train_size + test_size)
        splits.append((train_idx, test_idx))
        start += step_size
    return splits


def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """
    Calculate regression metrics for model evaluation.
    
    Returns dict with:
    - R²: Proportion of variance explained (1.0 = perfect, can be negative if worse than mean)
    - RMSE: Root Mean Squared Error in dollars (same units as y)
    - MAE: Mean Absolute Error in dollars (less sensitive to outliers than RMSE)
    - MAPE: Mean Absolute Percentage Error (the "headline" metric for MMM)
    - NRMSE: Normalized RMSE as % of mean (allows comparison across scales)
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_true != 0  # Avoid division by zero in MAPE
    
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.sum() > 0 else np.nan,
        'NRMSE': np.sqrt(mean_squared_error(y_true, y_pred)) / y_true.mean() * 100
    }

# Generate CV splits
cv_splits = time_series_cv_split(
    n_samples=len(y),
    train_size=config.cv_train_weeks,
    test_size=config.cv_test_weeks,
    step_size=config.cv_step_weeks
)

print(f"Time-Series Cross-Validation:")
print(f"  Training window: {config.cv_train_weeks} weeks")
print(f"  Test window: {config.cv_test_weeks} weeks")
print(f"  Number of folds: {len(cv_splits)}")


In [ ]:
# =============================================================================
# CELL 7: Hyperparameter Optimization with Nevergrad
# =============================================================================
#
# THE OPTIMIZATION PROBLEM:
#
# We need to find 3 hyperparameters PER CHANNEL:
#   - theta (adstock decay): How quickly does ad effect fade?
#   - alpha (saturation shape): How steep is the diminishing returns curve?
#   - gamma (half-saturation): At what spend level do we hit 50% of max response?
#
# With 20 channels, that's 60 parameters. We can't grid search (60^10 = impossible).
# We can't use gradient descent (the objective isn't smooth w.r.t. these params).
#
# SOLUTION: Evolutionary optimization (Nevergrad's TwoPointsDE)
#
# TwoPointsDE is a variant of Differential Evolution that:
#   1. Starts with a population of random parameter guesses
#   2. "Breeds" new guesses by combining good performers
#   3. Keeps the best, discards the worst
#   4. Repeats for `budget` iterations
#
# WHY 500 ITERATIONS: Empirically, loss stabilizes around 300-500 for this scale.
# Robyn uses 2000+ but also optimizes decomposition. We're simpler.
#
# SIGMOID REPARAMETRIZATION:
# Instead of letting optimizer search [0, 0.95] directly, we search [-5, 5] and
# apply sigmoid to map to the valid range. This is a standard trick to:
#   - Avoid boundary issues (optimizer doesn't get stuck at 0 or 0.95)
#   - Make the search space more "smooth" for the evolutionary algorithm
#
# NEGATIVE COEFFICIENT PENALTY:
# Economically, marketing should never HURT revenue. If the model wants to assign
# a negative coefficient (e.g., "more LinkedIn spend = less revenue"), we penalize
# this heavily. In practice, negative coefficients usually mean multicollinearity.
#
# ─────────────────────────────────────────────────────────────────────────────
# DEEPER DIVE: WHY DERIVATIVE-FREE OPTIMIZATION?
# ─────────────────────────────────────────────────────────────────────────────
#
# The objective function L(θ, α, γ) = 1 - R²(model trained with those params)
# involves FITTING A RIDGE REGRESSION INSIDE each evaluation. This creates
# a "nested" optimization that makes gradients unavailable or meaningless:
#
#   ∂L/∂θ = ??? (how does R² change if we nudge decay by 0.01?)
#
# The relationship is:
#   θ → adstock_transform(X) → Ridge.fit(X_transformed) → R²
#
# Each step involves discrete choices (which data points are "influential"),
# matrix inversions, and nonlinear transforms. Autodiff doesn't help here.
#
# TwoPointsDE (Differential Evolution variant):
# - Maintains a population of ~20-50 candidate solutions
# - Creates new candidates via: x_new = x_a + F·(x_b - x_c) + noise
#   where F is a mutation factor and a,b,c are random population members
# - "TwoPoints" variant uses 2 random points for crossover, improving
#   exploitation vs. exploration balance
# - No gradients needed—just function evaluations and selection
#
# Alternatives considered:
# - Bayesian Optimization: Better for <20 params, but O(n³) with iterations
# - Random Search: Surprisingly effective, but needs 10x more iterations
# - Hyperband: Good for early stopping, but our objective is fast to evaluate
#
# ─────────────────────────────────────────────────────────────────────────────
# RIDGE REGRESSION: WHY L2 PENALTY?
# ─────────────────────────────────────────────────────────────────────────────
#
# Standard OLS minimizes: ||y - Xβ||²
# Ridge adds:             ||y - Xβ||² + λ||β||²
#
# The L2 penalty (λ||β||²) does two things:
# 1. REGULARIZATION: Shrinks coefficients toward zero, preventing overfitting
#    when n (samples) is small relative to p (features). With 104 weeks and
#    20+ channels, we're in moderate-dimensional territory.
#
# 2. MULTICOLLINEARITY FIX: When channels are correlated (LinkedIn and Display
#    spike together in Q4), (X'X) is near-singular. Ridge adds λI to the diagonal:
#    β_ridge = (X'X + λI)⁻¹X'y, which is always invertible.
#
# Why NOT Lasso (L1)? Lasso sets some coefficients exactly to zero, which is
# great for feature selection but problematic here—we WANT every channel's
# contribution estimated, even if small. Zeroing LinkedIn would lose insights.
#
# Why NOT ElasticNet? Adds complexity without clear benefit for our use case.
# Ridge's closed-form solution is also computationally efficient.
# =============================================================================

class MMMOptimizer:
    """
    Marketing Mix Model optimizer using Nevergrad evolutionary algorithm.
    
    Finds optimal (theta, alpha, gamma) for each channel by minimizing (1 - R²)
    with a penalty for economically invalid negative coefficients.
    """
    
    def __init__(self, X_media, X_control, y, channels, config, observed_roas=None):
        self.X_media = X_media
        self.X_control = X_control
        self.y = y
        self.channels = channels
        self.config = config
        self.n_params = len(channels) * 3  # 3 params per channel
        # Store max spend per channel for gamma scaling
        self.channel_max = {ch: max(X_media[ch].max(), 1) for ch in channels}
        # Store observed ROAS for each channel (used in ROI constraint)
        self.observed_roas = observed_roas if observed_roas else {}
        
    def _decode_params(self, flat_params):
        """
        Decode flat parameter array into structured dict.
        
        Uses sigmoid transform to map unbounded search space [-5, 5] to valid ranges:
        - theta: [0, 0.95] (can't be 1.0 or adstock explodes)
        - alpha: [0.5, 3.0] (reasonable S-curve shapes)
        - gamma: [0, max_spend] (scaled to channel's observed range)
        """
        params = {}
        for i, ch in enumerate(self.channels):
            base = i * 3
            raw_theta, raw_alpha, raw_gamma = flat_params[base:base+3]
            
            # Sigmoid: 1/(1+e^-x) maps (-∞,∞) → (0,1), then scale to target range
            theta = 1 / (1 + np.exp(-raw_theta)) * 0.95  # [0, 0.95]
            alpha = 0.5 + 1 / (1 + np.exp(-raw_alpha)) * 2.5  # [0.5, 3.0]
            gamma = 1 / (1 + np.exp(-raw_gamma)) * self.channel_max[ch]  # [0, max]
            
            params[ch] = {'theta': theta, 'alpha': alpha, 'gamma': max(gamma, 1e-6)}
        return params
    
    def _objective(self, flat_params):
        """
        Objective function: Minimize (1 - R²) + penalty for negative coefficients + ROI constraint.
        
        Why (1 - R²)? We want to MAXIMIZE R², but optimizers MINIMIZE.
        So we minimize (1 - R²), which is 0 when R² = 1 (perfect fit).
        
        Why the penalties?
        1. Negative coefficient penalty: Marketing spend should never decrease revenue.
        2. ROI constraint penalty: Model ROI should be within reasonable range of observed ROAS.
           This prevents the model from assigning unrealistic attribution (e.g., 48x ROI on TikTok).
        """
        params = self._decode_params(flat_params)
        X_media_trans = apply_media_transformations(self.X_media, params, self.channels)
        X_full = pd.concat([X_media_trans, self.X_control], axis=1)
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_full)
        
        model = Ridge(alpha=self.config.ridge_alpha)
        model.fit(X_scaled, self.y)
        y_pred = model.predict(X_scaled)
        
        # Penalize negative media coefficients (economically invalid)
        media_coefs = model.coef_[:len(self.channels)] / scaler.scale_[:len(self.channels)]
        negative_penalty = np.sum(np.minimum(media_coefs, 0) ** 2) * 10
        
        # ROI constraint: penalize ROI estimates far from observed ROAS
        # This keeps model attribution grounded in reality
        # DEMO: Comment out this block (lines 161-174) to see unconstrained ROI estimates (e.g., 48x TikTok)
        roi_penalty = 0.0
        for i, ch in enumerate(self.channels):
            coef = media_coefs[i]
            contribution = X_media_trans[ch].sum() * coef
            spend = self.X_media[ch].sum()
            model_roi = contribution / spend if spend > 0 else 0
            observed_roas = self.observed_roas.get(ch, 1.0)
            
            # Allow 3x deviation from observed ROAS before penalty kicks in
            # DEMO: Change 3 to 10 for looser constraints, or 1.5 for tighter
            max_allowed_roi = max(observed_roas * 3, 5.0)  # At least 5x to allow some flexibility
            if model_roi > max_allowed_roi:
                roi_penalty += ((model_roi - max_allowed_roi) / max_allowed_roi) ** 2
        
        roi_penalty *= 5  # Scale penalty weight (DEMO: Set to 0 to disable ROI constraint entirely)
        
        r2 = r2_score(self.y, y_pred)
        return (1 - r2) + negative_penalty + roi_penalty
    
    def optimize(self, budget=500):
        """
        Run Nevergrad optimization.
        
        TwoPointsDE (Two-Points Differential Evolution):
        - Population-based evolutionary algorithm
        - Creates new candidates by combining existing good solutions
        - Robust to non-smooth, non-convex objective landscapes
        - 500 iterations typically sufficient for 50-100 parameters
        """
        print(f"\nOptimizing {self.n_params} parameters ({len(self.channels)} channels × 3 params)...")
        
        # Search space: unbounded, will be mapped via sigmoid in _decode_params
        parametrization = ng.p.Array(shape=(self.n_params,)).set_bounds(-5, 5)
        optimizer = ng.optimizers.TwoPointsDE(parametrization=parametrization, budget=budget)
        recommendation = optimizer.minimize(self._objective)
        
        best_params = self._decode_params(recommendation.value)
        final_loss = self._objective(recommendation.value)
        
        print(f"Optimization complete. Final loss: {final_loss:.4f} (R² ≈ {1 - final_loss:.4f})")
        return best_params, {'final_loss': final_loss}

# Calculate observed ROAS for each channel (ground truth to constrain model)
observed_roas = {}
for ch in channels:
    spend = X_media[ch].sum()
    # Revenue is proportionally allocated by spend share
    # Using y (total weekly revenue), we estimate channel contribution by spend proportion
    revenue = df[df['CHANNEL_KEY'] == ch]['REVENUE'].sum() if 'REVENUE' in df.columns else 0
    if spend > 0 and revenue > 0:
        observed_roas[ch] = revenue / spend
    else:
        observed_roas[ch] = 1.0  # Default to breakeven if no data
        
print(f"\nObserved ROAS by channel (ground truth for ROI constraints):")
for ch, roas in sorted(observed_roas.items(), key=lambda x: -x[1])[:10]:
    print(f"  {ch}: {roas:.2f}x")

# Run optimization with ROI constraints
optimizer = MMMOptimizer(X_media, X_control, y, channels, config, observed_roas=observed_roas)
best_params, opt_metrics = optimizer.optimize(budget=config.nevergrad_budget)

print(f"\nSample optimized parameters (first 5 channels):")
for ch in list(best_params.keys())[:5]:
    p = best_params[ch]
    print(f"  {ch}: theta={p['theta']:.3f}, alpha={p['alpha']:.3f}, gamma={p['gamma']:.1f}")


In [ ]:
# =============================================================================
# CELL 8: Final Model Training with Cross-Validation Metrics
# =============================================================================
#
# TWO SEPARATE FITS:
#
# 1. IN-SAMPLE FIT (Full Data):
#    - Train on ALL data, predict on ALL data
#    - R² will be high (0.90+) because model has "seen" every data point
#    - Use for: coefficient interpretation, response curves, budget optimization
#
# 2. CROSS-VALIDATION FIT (Rolling Window):
#    - Train on past, predict on future, repeat
#    - R² and MAPE will be WORSE than in-sample (this is expected!)
#    - Use for: realistic accuracy estimate, "will this work in production?"
#
# WHY REPORT BOTH:
#   - If in-sample R² = 0.95 but CV R² = 0.50, model is OVERFITTING
#     (memorizing training data, not learning generalizable patterns)
#   - Healthy gap: in-sample R² ≈ CV R² + 0.05-0.10
#   - If gap is large: reduce model complexity (fewer channels, stronger ridge penalty)
#
# QUALITY THRESHOLDS (industry standard):
#   CV MAPE < 10%: Excellent - model is highly predictive
#   CV MAPE 10-20%: Good - suitable for budget optimization
#   CV MAPE 20-30%: Acceptable - directional insights only
#   CV MAPE > 30%: Poor - investigate data quality or model specification
#
# ─────────────────────────────────────────────────────────────────────────────
# DEEPER DIVE: BIAS-VARIANCE TRADEOFF IN MMM
# ─────────────────────────────────────────────────────────────────────────────
#
# The gap between in-sample and CV performance illustrates the bias-variance
# tradeoff central to all statistical learning:
#
#   Expected Error = Bias² + Variance + Irreducible Noise
#
# BIAS: Systematic error from model assumptions (e.g., assuming linearity when
#       relationships are nonlinear). High bias = underfitting.
#
# VARIANCE: Sensitivity to the specific training data. A complex model might
#           fit training data perfectly but fail on new data. High variance = overfitting.
#
# IN-SAMPLE ERROR captures (mostly) bias: if the model can't fit the training
# data well, it has too little flexibility. Low in-sample error means the model
# can represent the data's complexity.
#
# CV ERROR captures bias + variance: on unseen data, both systematic errors AND
# overfitting manifest. The GAP between in-sample and CV is (roughly) variance.
#
# HOW RIDGE HELPS:
# Ridge penalty λ||β||² adds bias (shrinks coefficients toward zero) but reduces
# variance (prevents wild coefficient estimates from correlated features).
#
# If CV performance is poor despite good in-sample fit:
#   - Increase λ (more regularization) to reduce variance
#   - Reduce model complexity (fewer channels, simpler transforms)
#   - Get more data (especially more time periods)
#
# If both in-sample AND CV are poor:
#   - Model may be too simple (underfitting)
#   - Check data quality (missing spend, misaligned time series)
#   - Consider adding control variables (macro factors, seasonality)
#
# THE ±STD IN CV METRICS:
# We report mean ± std across folds. High std indicates inconsistent performance
# across time periods. This could mean:
#   - Some quarters are inherently harder to predict (Q4 chaos)
#   - Structural breaks (pandemic, new product launch)
#   - Concept drift (marketing effectiveness changing over time)
# =============================================================================

def train_final_model(X_media, X_control, y, channels, params, cv_splits, config):
    """
    Train final model and compute both in-sample and cross-validation metrics.
    
    In-sample metrics show model fit; CV metrics show predictive accuracy.
    Large gap between them indicates overfitting.
    """
    # Transform media with optimized hyperparameters
    X_media_trans = apply_media_transformations(X_media, params, channels)
    X_full = pd.concat([X_media_trans, X_control], axis=1)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_full)
    
    # In-Sample Fit
    model = Ridge(alpha=config.ridge_alpha)
    model.fit(X_scaled, y)
    y_pred_insample = model.predict(X_scaled)
    insample_metrics = calculate_metrics(y.values, y_pred_insample)
    
    # Cross-Validation
    cv_metrics_list = []
    y_values = y.values
    
    for fold_idx, (train_idx, test_idx) in enumerate(cv_splits):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y_values[train_idx], y_values[test_idx]
        
        cv_model = Ridge(alpha=config.ridge_alpha)
        cv_model.fit(X_train, y_train)
        y_pred = cv_model.predict(X_test)
        
        fold_metrics = calculate_metrics(y_test, y_pred)
        fold_metrics['fold'] = fold_idx + 1
        cv_metrics_list.append(fold_metrics)
    
    cv_metrics_df = pd.DataFrame(cv_metrics_list)
    cv_metrics_avg = cv_metrics_df.drop('fold', axis=1).mean().to_dict()
    cv_metrics_std = cv_metrics_df.drop('fold', axis=1).std().to_dict()
    
    metrics = {
        'in_sample': insample_metrics,
        'cv_mean': cv_metrics_avg,
        'cv_std': cv_metrics_std
    }
    
    return model, scaler, X_full, metrics

# Train final model
model, scaler, X_transformed, metrics = train_final_model(
    X_media, X_control, y, channels, best_params, cv_splits, config
)

# Display metrics
print("\n" + "="*60)
print("MODEL PERFORMANCE METRICS")
print("="*60)

print("\nIn-Sample (Full Data):")
for metric, value in metrics['in_sample'].items():
    print(f"  {metric}: {value:.4f}")

print("\nCross-Validation (Out-of-Sample):")
for metric in ['R2', 'MAPE', 'NRMSE']:
    mean_val = metrics['cv_mean'].get(metric, 0)
    std_val = metrics['cv_std'].get(metric, 0)
    print(f"  {metric}: {mean_val:.4f} ± {std_val:.4f}")

# Model quality assessment
cv_mape = metrics['cv_mean'].get('MAPE', 100)
if cv_mape < 10:
    quality = "EXCELLENT"
elif cv_mape < 20:
    quality = "GOOD"
elif cv_mape < 30:
    quality = "ACCEPTABLE"
else:
    quality = "NEEDS IMPROVEMENT"

print(f"\nModel Quality: {quality} (CV MAPE = {cv_mape:.1f}%)")


In [ ]:
# =============================================================================
# CELL 9: Bootstrap Confidence Intervals for ROI
# =============================================================================
#
# WHY BOOTSTRAP INSTEAD OF ANALYTIC CONFIDENCE INTERVALS:
#
# Traditional CI formulas assume:
#   - Normally distributed errors
#   - Independent observations
#   - Linear relationships
#
# MMM violates all three:
#   - Errors are often heteroscedastic (bigger in Q4)
#   - Time series has autocorrelation (this week's revenue predicts next week's)
#   - We applied nonlinear transforms (adstock, saturation)
#
# BOOTSTRAP APPROACH:
#   1. Resample the data WITH REPLACEMENT (some weeks appear twice, some not at all)
#   2. Re-fit the model on this "fake" dataset
#   3. Calculate ROI for each channel
#   4. Repeat 100 times
#   5. The 5th and 95th percentiles of these 100 ROIs = 90% confidence interval
#
# INTERPRETATION:
#   - ROI = 3.2 [2.8, 3.6] means: "We estimate LinkedIn returns $3.20 per dollar,
#     and we're 90% confident the true value is between $2.80 and $3.60"
#   - IS_SIGNIFICANT = True means the entire CI is above zero (we're confident
#     the channel has positive ROI, not just statistical noise)
#
# NOTE: We keep adstock/saturation params FIXED during bootstrap. We're quantifying
# uncertainty in the COEFFICIENTS, not the hyperparameters. Full uncertainty would
# require nested optimization (computationally prohibitive).
#
# ─────────────────────────────────────────────────────────────────────────────
# DEEPER DIVE: THE BOOTSTRAP PRINCIPLE
# ─────────────────────────────────────────────────────────────────────────────
#
# The key insight (Efron, 1979): The empirical distribution of your sample
# approximates the true population distribution. By resampling FROM your sample,
# you simulate what WOULD happen if you could re-run the entire data collection.
#
# For a statistic θ̂ (like ROI), the bootstrap distribution of θ̂* approximates
# the sampling distribution of θ̂. The standard error of θ̂* over B bootstrap
# samples estimates the true standard error of θ̂.
#
# PERCENTILE METHOD (what we use):
# The (α/2, 1-α/2) percentiles of the bootstrap distribution give a (1-α) CI.
# For 90% CI: we take the 5th and 95th percentiles of 100 bootstrap ROIs.
#
# ALTERNATIVE METHODS (not used here, but worth knowing):
# - BCa (Bias-Corrected Accelerated): Adjusts for skewness and bias in θ̂
# - Studentized Bootstrap: Divides by bootstrap SE, more accurate for small n
# - Block Bootstrap: For time series—resamples contiguous blocks to preserve
#   autocorrelation. We don't use this because our primary goal is coefficient
#   uncertainty, and time-series structure is less critical for that.
#
# WHY 100 ITERATIONS:
# - SE of a percentile estimate ≈ √(p(1-p)/B) where p is the percentile
# - For p=0.05 and B=100: SE ≈ 0.022 (good enough for practical decisions)
# - B=1000 would give SE ≈ 0.007 (diminishing returns for 10x compute)
#
# COEFFICIENT UNSCALING:
# Note: We divide by scaler.scale_ to convert back to original units.
# StandardScaler transforms: X_scaled = (X - μ) / σ
# Ridge fits: y = Σ β_scaled[i] · X_scaled[i]
# To get interpretable coefficients: β_original[i] = β_scaled[i] / σ[i]
# =============================================================================

def bootstrap_roi_confidence(X_media, X_control, y, channels, params, config):
    """
    Bootstrap confidence intervals for channel ROI estimates.
    
    Resamples data 100 times, re-fits model each time, collects distribution
    of ROI estimates. This captures uncertainty from:
      - Sample variability (different weeks have different patterns)
      - Coefficient estimation (regression has standard errors)
    
    Does NOT capture uncertainty in hyperparameters (theta, alpha, gamma).
    """
    n_samples = len(y)
    n_bootstrap = config.n_bootstrap
    ci_level = config.confidence_level
    
    # Apply transformations once (params are fixed)
    X_media_trans = apply_media_transformations(X_media, params, channels)
    X_full = pd.concat([X_media_trans, X_control], axis=1)
    
    roi_samples = {ch: [] for ch in channels}
    coef_samples = {ch: [] for ch in channels}
    
    print(f"\nRunning {n_bootstrap} bootstrap iterations for {ci_level*100:.0f}% CI...")
    
    for b in range(n_bootstrap):
        boot_idx = np.random.choice(n_samples, size=n_samples, replace=True)
        X_boot = X_full.iloc[boot_idx]
        y_boot = y.iloc[boot_idx]
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_boot)
        
        model = Ridge(alpha=config.ridge_alpha)
        model.fit(X_scaled, y_boot)
        
        coefs = model.coef_ / scaler.scale_
        
        for i, ch in enumerate(channels):
            coef = coefs[i]
            coef_samples[ch].append(coef)
            
            contribution = X_media_trans[ch].iloc[boot_idx].sum() * coef
            spend = X_media[ch].iloc[boot_idx].sum()
            roi = contribution / spend if spend > 0 else 0
            roi_samples[ch].append(roi)
        
        if (b + 1) % 25 == 0:
            print(f"  Completed {b + 1}/{n_bootstrap}")
    
    # Calculate confidence intervals
    alpha = 1 - ci_level
    results = []
    
    for ch in channels:
        rois = np.array(roi_samples[ch])
        coefs = np.array(coef_samples[ch])
        ci_key = f'ROI_CI_{int(ci_level*100)}'
        
        results.append({
            'CHANNEL_KEY': ch,
            'ROI_MEAN': np.mean(rois),
            'ROI_MEDIAN': np.median(rois),
            'ROI_STD': np.std(rois),
            f'ROI_CI_LOWER_{int(ci_level*100)}': np.percentile(rois, alpha/2 * 100),
            f'ROI_CI_UPPER_{int(ci_level*100)}': np.percentile(rois, (1 - alpha/2) * 100),
            'COEF_MEAN': np.mean(coefs),
            'COEF_STD': np.std(coefs),
            'TOTAL_SPEND': X_media[ch].sum()
        })
    
    roi_ci = pd.DataFrame(results)
    roi_ci['IS_SIGNIFICANT'] = roi_ci[f'ROI_CI_LOWER_{int(ci_level*100)}'] > 0
    
    return roi_ci.sort_values('ROI_MEAN', ascending=False)

# Run bootstrap
roi_confidence = bootstrap_roi_confidence(
    X_media, X_control, y, channels, best_params, config
)

print("\n" + "="*60)
print("CHANNEL ROI WITH CONFIDENCE INTERVALS")
print("="*60)
print(f"\nConfidence Level: {config.confidence_level*100:.0f}%")
print("\nTop 10 Channels by ROI:")
display_cols = ['CHANNEL_KEY', 'ROI_MEAN', f'ROI_CI_LOWER_{int(config.confidence_level*100)}', 
                f'ROI_CI_UPPER_{int(config.confidence_level*100)}', 'IS_SIGNIFICANT', 'TOTAL_SPEND']
print(roi_confidence[display_cols].head(10).to_string(index=False))


In [ ]:
# =============================================================================
# CELL 10: Generate Response Curves and Marginal ROI
# =============================================================================
#
# RESPONSE CURVES: THE KEY VISUALIZATION
#
# A response curve shows predicted revenue contribution as a function of spend:
#
#   Revenue │              ●●●●●●●●●●●●  ← Saturation zone (mROI < 1)
#   Contrib │         ●●●●●               Each extra $ returns < $1
#     ▲     │      ●●●                    = "Wasted spend"
#     │     │    ●●
#     │     │  ●●                       ← Efficient zone (mROI > 1)
#     │     │●●                           Each extra $ returns > $1
#     └─────┴──────────────────────────► Spend
#           0      $50k     $100k
#
# AVERAGE ROI vs. MARGINAL ROI:
#
#   Average ROI = Total contribution / Total spend
#               = "What did we get back per dollar HISTORICALLY?"
#
#   Marginal ROI = d(contribution)/d(spend) at CURRENT spend level
#                = "What will we get back for the NEXT dollar?"
#
# MARGINAL ROI IS MORE USEFUL because:
#   - A channel with 5x average ROI might be saturated (0.5x marginal)
#   - A channel with 2x average ROI might have headroom (3x marginal)
#   - Budget decisions are about the NEXT dollar, not past dollars
#
# HOW WE CALCULATE MARGINAL ROI:
#   Numerical derivative: [f(x + δ) - f(x)] / δ
#   where f(x) = coefficient × hill_saturation(adstock(x))
#
# ─────────────────────────────────────────────────────────────────────────────
# DEEPER DIVE: THE CALCULUS OF MARGINAL ROI
# ─────────────────────────────────────────────────────────────────────────────
#
# The full response function is a composition of three transforms:
#
#   R(s) = β · H(A(s))
#
# where:
#   s = weekly spend
#   A(s) = s/(1-θ) = steady-state adstock (geometric series limit)
#   H(x) = x^α / (x^α + γ^α) = Hill saturation function
#   β = regression coefficient (revenue per unit of saturated adstock)
#
# The marginal ROI is dR/ds, applying chain rule:
#
#   dR/ds = β · dH/dA · dA/ds
#
#   dA/ds = 1/(1-θ)  (linear relationship at steady state)
#
#   dH/dA = α·γ^α·A^(α-1) / (A^α + γ^α)²  (Hill function derivative)
#
# Substituting:
#   mROI = β · [α·γ^α·A^(α-1) / (A^α + γ^α)²] · [1/(1-θ)]
#
# KEY INSIGHT: As A → ∞ (high spend), (A^α + γ^α)² grows as A^(2α), but the
# numerator only grows as A^(α-1). Net effect: mROI → 0 as spend → ∞.
# This is diminishing returns made explicit through calculus.
#
# AT THE HALF-SATURATION POINT (A = γ):
#   H(γ) = 0.5, and dH/dA = α/(4γ)
#   mROI = β · α/(4γ) · 1/(1-θ)
#
# This gives us a quick diagnostic: if mROI at current spend is close to
# α·β/(4γ(1-θ)), we're roughly at the "efficient frontier" of the S-curve.
#
# NUMERICAL VS. ANALYTIC DERIVATIVE:
# We use numerical differentiation [f(x+δ) - f(x)]/δ for simplicity and to
# match what the optimizer actually "sees". Analytic form above is for intuition.
#
# EFFICIENCY ZONE THRESHOLDS:
#   mROI > 1.5: EFFICIENT - Every dollar returns >$1.50, strong investment
#   mROI 0.8-1.5: DIMINISHING - Still positive but flattening
#   mROI < 0.8: SATURATED - Likely better to reallocate to other channels
#
# These thresholds are heuristics, not physical laws. Adjust based on your
# cost of capital and strategic priorities.
# =============================================================================

def generate_response_curves(X_media, channels, params, coefficients, roi_confidence, n_points=100):
    """
    Generate response curves with confidence intervals and efficiency zones.
    
    Response curves show the spend → revenue relationship for each channel.
    Marginal ROI is the slope of this curve at current spend level.
    
    ENHANCED OUTPUT INCLUDES:
    - CI bands: Upper/lower predictions based on bootstrap coefficient variance
    - Marginal ROI at each point: Answers "what's the next dollar worth HERE?"
    - Efficiency zone: EFFICIENT (mROI > 1.5), DIMINISHING (0.8-1.5), SATURATED (< 0.8)
    """
    curves = []
    marginal_roi = {}
    
    # Re-fit to get current coefficients
    X_media_trans = apply_media_transformations(X_media, params, channels)
    X_full = pd.concat([X_media_trans, X_control], axis=1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_full)
    model.fit(X_scaled, y)
    coefficients = dict(zip(channels, model.coef_[:len(channels)] / scaler.scale_[:len(channels)]))
    
    # Get coefficient uncertainty from bootstrap (for CI bands)
    coef_std = {}
    for ch in channels:
        if roi_confidence is not None and 'COEF_STD' in roi_confidence.columns:
            ch_row = roi_confidence[roi_confidence['CHANNEL_KEY'] == ch]
            coef_std[ch] = ch_row['COEF_STD'].values[0] if len(ch_row) > 0 else 0
        else:
            coef_std[ch] = coefficients.get(ch, 0) * 0.15  # Default 15% uncertainty
    
    for ch in channels:
        p = params[ch]
        coef = coefficients.get(ch, 0)
        coef_uncertainty = coef_std.get(ch, 0)
        
        current_spend = X_media[ch].mean()
        max_spend = X_media[ch].max() * 3
        gamma = p['gamma']  # Half-saturation point
        
        spend_range = np.linspace(0, max_spend, n_points)
        
        for i, spend in enumerate(spend_range):
            adstock_steady = spend / (1 - p['theta']) if p['theta'] < 1 else spend
            saturated = hill_saturation(np.array([adstock_steady]), p['alpha'], p['gamma'])[0]
            contribution = saturated * coef
            
            # CI bands (scale coefficient uncertainty through saturation transform)
            contribution_ci_lower = saturated * max(0, coef - 1.645 * coef_uncertainty)
            contribution_ci_upper = saturated * (coef + 1.645 * coef_uncertainty)
            
            # Marginal ROI at this spend level (numerical derivative)
            delta = max(spend * 0.01, 100)  # At least $100 increment
            adstock_delta = (spend + delta) / (1 - p['theta']) if p['theta'] < 1 else (spend + delta)
            response_delta = hill_saturation(np.array([adstock_delta]), p['alpha'], p['gamma'])[0] * coef
            marginal_roi_at_spend = (response_delta - contribution) / delta if delta > 0 else 0
            
            # Classify efficiency zone based on marginal ROI
            if marginal_roi_at_spend > 1.5:
                zone = 'EFFICIENT'
            elif marginal_roi_at_spend >= 0.8:
                zone = 'DIMINISHING'
            else:
                zone = 'SATURATED'
            
            curves.append({
                'CHANNEL': ch,
                'SPEND': spend,
                'PREDICTED_REVENUE': contribution,
                'PREDICTED_REVENUE_CI_LOWER': contribution_ci_lower,
                'PREDICTED_REVENUE_CI_UPPER': contribution_ci_upper,
                'MARGINAL_ROI_AT_SPEND': marginal_roi_at_spend,
                'EFFICIENCY_ZONE': zone
            })
        
        # Marginal ROI at current spend (for summary)
        delta = current_spend * 0.01
        adstock_curr = current_spend / (1 - p['theta']) if p['theta'] < 1 else current_spend
        response_curr = hill_saturation(np.array([adstock_curr]), p['alpha'], p['gamma'])[0] * coef
        
        adstock_delta = (current_spend + delta) / (1 - p['theta']) if p['theta'] < 1 else (current_spend + delta)
        response_delta = hill_saturation(np.array([adstock_delta]), p['alpha'], p['gamma'])[0] * coef
        
        marginal_roi[ch] = (response_delta - response_curr) / delta if delta > 0 else 0
    
    return pd.DataFrame(curves), marginal_roi

# Generate curves with CI bands
response_curves, marginal_roi = generate_response_curves(X_media, channels, best_params, {}, roi_confidence)

print("\n" + "="*60)
print("MARGINAL ROI (Value of Next Dollar Spent)")
print("="*60)
print("\nTop 10 Channels by Marginal ROI:")
marginal_df = pd.DataFrame([
    {'CHANNEL_KEY': ch, 'MARGINAL_ROI': roi} 
    for ch, roi in marginal_roi.items()
]).sort_values('MARGINAL_ROI', ascending=False)
print(marginal_df.head(10).to_string(index=False))
print(f"\nResponse curves generated: {len(response_curves)} data points")


In [ ]:
# =============================================================================
# CELL 11: Budget Optimizer
# =============================================================================
#
# THE BUDGET ALLOCATION PROBLEM:
#
# Given the learned response curves, how should we reallocate spend to maximize
# total revenue? This is a constrained optimization problem:
#
#   MAXIMIZE: Total predicted revenue = Σ (saturated_response × coefficient)
#   SUBJECT TO:
#     1. Total budget unchanged (budget neutral)
#     2. Each channel can only change ±30% (realistic for CMO approval)
#     3. All spend >= 0 (can't have negative spend)
#
# WHY CONSTRAINTS MATTER:
#
# Without constraints, the optimizer would say "put 100% in LinkedIn" because
# it has the highest marginal ROI. But this is impractical:
#   - CMOs can't pivot all spend in one quarter
#   - Vendor contracts require minimum commitments
#   - Channel capacity limits exist (LinkedIn inventory is finite)
#
# The ±30% limit keeps recommendations actionable. If LinkedIn is at $1M/quarter,
# we recommend up to $1.3M, not $10M.
#
# SLSQP ALGORITHM:
# Sequential Least Squares Programming - a constrained optimizer that handles
# both equality constraints (total budget) and inequality constraints (±30%).
# Faster than evolutionary methods for smooth, convex problems like this.
#
# PREDICTED LIFT:
# The "predicted lift" is the difference between:
#   - Current revenue (with current allocation)
#   - Optimized revenue (with recommended allocation)
# This is the "headline number" for the CMO: "Shifting spend could add $2.4M"
#
# ─────────────────────────────────────────────────────────────────────────────
# DEEPER DIVE: CONSTRAINED OPTIMIZATION FORMULATION
# ─────────────────────────────────────────────────────────────────────────────
#
# We work in PROPORTION space (x_i = spend_i / total_budget) for numerical
# stability and cleaner constraint expression.
#
# FORMAL PROBLEM:
#   min  -Σ f_i(x_i · B)      (we negate because scipy.minimize minimizes)
#   s.t. Σ x_i = 1            (equality: budget-neutral)
#        (1-δ)·x_i^0 ≤ x_i ≤ (1+δ)·x_i^0  for all i  (inequality: ±δ bounds)
#        x_i ≥ 0              for all i  (implicit in bounds)
#
# where:
#   f_i(s) = β_i · H_i(A_i(s))  is the response function for channel i
#   B = total budget
#   x_i^0 = current proportion for channel i
#   δ = 0.30 (our 30% change limit)
#
# WHY SLSQP (Sequential Least Squares Quadratic Programming):
# 1. Handles both equality (budget) and inequality (bounds) constraints
# 2. Uses quadratic approximation of the Lagrangian at each step
# 3. Convergence is typically fast for smooth, moderately-sized problems
# 4. Available in scipy.optimize.minimize with method='SLSQP'
#
# The Lagrangian for our problem:
#   L(x, λ, μ) = -Σ f_i(x_i·B) + λ·(Σx_i - 1) + Σ μ_i·(constraint violations)
#
# At the optimum, KKT conditions require:
#   ∂f_i/∂x_i = λ for all active channels (marginal returns equalized!)
#
# ECONOMIC INTERPRETATION:
# The optimal allocation EQUALIZES MARGINAL ROI across all channels (subject
# to constraints). This is the principle of marginal analysis: reallocate from
# low-mROI to high-mROI channels until they're equal.
#
# If channel A has mROI = 3.0 and channel B has mROI = 1.5, we should shift
# budget A→B until they converge (typically around 2.0 for both).
#
# The ±30% constraint prevents extreme shifts, so post-optimization mROIs
# won't be perfectly equal—but they'll be closer than the starting point.
#
# CAUTION ON "PREDICTED LIFT":
# The lift estimate assumes the model is correctly specified and extrapolates
# well. In practice, treat it as directional. A "10% lift" doesn't mean you'll
# get exactly 10%—it means reallocation is likely beneficial.
# =============================================================================

def optimize_budget(X_media, channels, params, marginal_roi, budget_change_limit=0.30):
    """
    Optimize budget allocation to maximize predicted revenue.
    
    Uses constrained optimization (SLSQP) to find the best reallocation
    within business constraints (±30% per channel, budget neutral).
    """
    current_spend = {ch: X_media[ch].sum() for ch in channels}
    total_budget = sum(current_spend.values())
    
    # Only optimize channels with actual spend (avoid divide-by-zero)
    active_channels = [ch for ch in channels if current_spend[ch] > 0]
    n_channels = len(active_channels)
    
    if n_channels == 0:
        return pd.DataFrame()
    
    # Re-fit model to get coefficients (needed for revenue prediction)
    X_media_trans = apply_media_transformations(X_media, params, channels)
    X_full = pd.concat([X_media_trans, X_control], axis=1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_full)
    model.fit(X_scaled, y)
    # Unscale coefficients to get "per-unit" impact
    coefficients = dict(zip(channels, model.coef_[:len(channels)] / scaler.scale_[:len(channels)]))
    
    x0 = np.array([current_spend[ch] / total_budget for ch in active_channels])
    
    def objective(x):
        total_contrib = 0
        for i, ch in enumerate(active_channels):
            spend = x[i] * total_budget
            p = params[ch]
            coef = coefficients.get(ch, 0)
            
            weekly_spend = spend / len(X_media)
            adstock = weekly_spend / (1 - p['theta']) if p['theta'] < 1 else weekly_spend
            saturated = hill_saturation(np.array([adstock]), p['alpha'], p['gamma'])[0]
            contribution = saturated * coef * len(X_media)
            total_contrib += contribution
        
        return -total_contrib
    
    budget_constraint = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}
    
    bounds = []
    for i, ch in enumerate(active_channels):
        current_prop = x0[i]
        lower = max(0, current_prop * (1 - budget_change_limit))
        upper = current_prop * (1 + budget_change_limit)
        bounds.append((lower, upper))
    
    result = minimize(objective, x0, method='SLSQP', bounds=bounds, 
                     constraints=budget_constraint, options={'maxiter': 1000})
    
    results = []
    for i, ch in enumerate(active_channels):
        current = current_spend[ch]
        recommended = result.x[i] * total_budget
        change = recommended - current
        change_pct = change / current * 100 if current > 0 else 0
        
        results.append({
            'CHANNEL_KEY': ch,
            'CURRENT_SPEND': current,
            'RECOMMENDED_SPEND': recommended,
            'CHANGE_AMOUNT': change,
            'CHANGE_PCT': change_pct,
            'MARGINAL_ROI': marginal_roi.get(ch, 0)
        })
    
    opt_df = pd.DataFrame(results).sort_values('CHANGE_PCT', ascending=False)
    
    current_contribution = -objective(x0)
    optimized_contribution = -objective(result.x)
    predicted_lift = optimized_contribution - current_contribution
    
    print(f"\nPredicted Revenue Lift: ${predicted_lift:,.0f}")
    print(f"Lift Percentage: {predicted_lift / current_contribution * 100:.1f}%")
    
    return opt_df

# Run budget optimization
print("\n" + "="*60)
print("BUDGET OPTIMIZATION RECOMMENDATIONS")
print("="*60)
print(f"\nConstraints: Budget neutral, ±{config.budget_change_limit*100:.0f}% per channel")

budget_recommendations = optimize_budget(
    X_media, channels, best_params, marginal_roi, config.budget_change_limit
)

print("\nTop 5 Channels to INCREASE:")
print(budget_recommendations.head(5)[['CHANNEL_KEY', 'CURRENT_SPEND', 'RECOMMENDED_SPEND', 'CHANGE_PCT']].to_string(index=False))

print("\nTop 5 Channels to DECREASE:")
print(budget_recommendations.tail(5)[['CHANNEL_KEY', 'CURRENT_SPEND', 'RECOMMENDED_SPEND', 'CHANGE_PCT']].to_string(index=False))


In [ ]:
# =============================================================================
# CELL 12: Prepare Results with Dimensional Keys
# =============================================================================
#
# OUTPUT STRUCTURE:
#
# The notebook produces granular ROI at the Channel × Region × Product level.
# Example: "LINKEDIN_EMEA_SI" → LinkedIn in EMEA for Safety & Industrial.
#
# We parse this back to separate columns so results can JOIN to dimension tables:
#   - CHANNEL_CODE → joins to MARKETING_CHANNEL dimension
#   - GEO_CODE → joins to GEOGRAPHY dimension (at configured level)
#   - PRODUCT_CODE → joins to PRODUCT_CATEGORY dimension (at configured level)
#
# This enables the Streamlit app to filter/slice results by any dimension
# without string parsing in SQL.
#
# KEY OUTPUT COLUMNS:
#   - ROI: Average historical return (contribution / spend)
#   - ROI_CI_LOWER/UPPER: Bootstrap confidence bounds
#   - MARGINAL_ROI: Return on NEXT dollar (derivative of response curve)
#   - OPTIMAL_SPEND_SUGGESTION: Budget optimizer recommendation
#   - ADSTOCK_DECAY_RATE: Learned theta (how quickly effect fades)
#   - SATURATION_POINT: Learned gamma (spend level at 50% of max response)
# =============================================================================

def parse_channel_key(channel_key):
    """
    Parse composite channel key back to dimensions.
    E.g., "LINKEDIN_EMEA_SI" → {CHANNEL: LINKEDIN, GEO: EMEA, PRODUCT: SI}
    """
    parts = channel_key.split('_')
    if len(parts) >= 3:
        return {'CHANNEL': parts[0], 'GEO': parts[1], 'PRODUCT': parts[2]}
    elif len(parts) == 2:
        return {'CHANNEL': parts[0], 'GEO': parts[1], 'PRODUCT': 'ALL'}
    else:
        return {'CHANNEL': parts[0] if parts else 'UNKNOWN', 'GEO': 'ALL', 'PRODUCT': 'ALL'}


def prepare_model_results(roi_confidence, marginal_roi, budget_recommendations, params, config, metrics, X_media):
    """
    Prepare final results DataFrame for saving to MMM.MODEL_RESULTS.
    
    ENHANCED OUTPUT now includes:
    - Full uncertainty quantification (CI bounds, significance)
    - Learned MMM parameters (adstock decay, saturation shape/scale)
    - Model quality metrics (R², CV MAPE)
    - Spend context (current spend, share of budget)
    """
    
    results = []
    ci_level = int(config.confidence_level * 100)
    
    # Calculate total spend across all channels for share calculation
    total_spend_all = sum(row['TOTAL_SPEND'] for _, row in roi_confidence.iterrows())
    
    for _, row in roi_confidence.iterrows():
        ch = row['CHANNEL_KEY']
        dims = parse_channel_key(ch)
        p = params.get(ch, {'theta': 0, 'alpha': 1, 'gamma': 1})
        
        budget_row = budget_recommendations[budget_recommendations['CHANNEL_KEY'] == ch]
        optimal_spend = budget_row['RECOMMENDED_SPEND'].values[0] if len(budget_row) > 0 else row['TOTAL_SPEND']
        
        # Calculate spend share
        spend_share = row['TOTAL_SPEND'] / total_spend_all if total_spend_all > 0 else 0
        
        # Count observations for this channel
        n_obs = len(X_media[ch].dropna()) if ch in X_media.columns else 0
        
        results.append({
            # Identifiers
            'MODEL_RUN_DATE': datetime.now().strftime('%Y-%m-%d'),
            'MODEL_VERSION': config.model_version,
            'CHANNEL_CODE': dims['CHANNEL'],
            'GEO_CODE': dims['GEO'],
            'PRODUCT_CODE': dims['PRODUCT'],
            'CHANNEL_KEY': ch,
            
            # Core metrics
            'COEFFICIENT_WEIGHT': row['COEF_MEAN'],
            'ROI': row['ROI_MEAN'],
            'MARGINAL_ROI': marginal_roi.get(ch, 0),
            
            # Confidence intervals (90% CI from bootstrap)
            'ROI_CI_LOWER': row[f'ROI_CI_LOWER_{ci_level}'],
            'ROI_CI_UPPER': row[f'ROI_CI_UPPER_{ci_level}'],
            'IS_SIGNIFICANT': row['IS_SIGNIFICANT'],
            
            # Learned MMM parameters
            'ADSTOCK_DECAY_RATE': p['theta'],
            'SATURATION_ALPHA': p['alpha'],  # Hill shape parameter
            'SATURATION_POINT': p['gamma'],   # Half-saturation spend level
            
            # Model quality
            'MODEL_R2_INSAMPLE': metrics['in_sample']['R2'],
            'MODEL_MAPE_CV': metrics['cv_mean'].get('MAPE', None),
            'N_OBSERVATIONS': n_obs,
            
            # Spend context
            'CURRENT_SPEND': row['TOTAL_SPEND'],
            'SPEND_SHARE': spend_share,
            'OPTIMAL_SPEND_SUGGESTION': optimal_spend
        })
    
    return pd.DataFrame(results)

# Prepare results with enhanced fields
model_results = prepare_model_results(
    roi_confidence, marginal_roi, budget_recommendations, best_params, config, metrics, X_media
)

print("\n" + "="*60)
print("MODEL RESULTS SUMMARY")
print("="*60)
print(f"\nTotal records: {len(model_results)}")
print(f"Model version: {config.model_version}")
print(f"\nColumns: {model_results.columns.tolist()}")
model_results.head()


In [ ]:
# =============================================================================
# CELL 13b: ROI Analysis with Spend Context
# =============================================================================
#
# KEY INSIGHT: ROI estimates are only reliable for channels with sufficient spend.
# Low-spend channels have inflated/uncertain ROI due to signal-to-noise issues.
#
# This cell provides:
# 1. ROI by Channel (all channels) - shows raw model output with spend labels
# 2. ROI vs Spend scatter - reveals the spend-reliability relationship
# 3. Filtered ROI view - user can exclude low-spend channels via slider
#
# The story: "High-spend channels have reliable ROI estimates. Low-spend channels
# need additional validation through incrementality testing."
# =============================================================================

import streamlit as st

# Color palette for consistent styling (matches Streamlit app)
COLORS = {
    'primary': '#0068C9',      # Snowflake blue
    'secondary': '#11567F',    # Deep blue  
    'positive': '#28A745',     # Green for positive metrics
    'negative': '#DC3545',     # Red for negative metrics
    'neutral': '#6B7280',      # Medium gray
    'accent': '#D95F02',       # Orange highlight
    'warning': '#F39C12'       # Amber for caution
}

ci_level = int(config.confidence_level * 100)

# ─────────────────────────────────────────────────────────────────────────────
# PANEL 1: ROI by Channel (All Channels) with Spend Labels
# ─────────────────────────────────────────────────────────────────────────────
st.subheader("1. ROI by Channel (All Channels)")
st.caption("⚠️ Note: Low-spend channels may show inflated ROI due to limited signal")

roi_chart_data = roi_confidence.sort_values('ROI_MEAN', ascending=True).tail(15)

# Add spend info to labels for context
roi_chart_data = roi_chart_data.copy()
roi_chart_data['LABEL'] = roi_chart_data.apply(
    lambda r: f"{r['CHANNEL_KEY']} (${r['TOTAL_SPEND']/1e6:.0f}M)", axis=1
)

fig_roi = go.Figure()

fig_roi.add_trace(go.Bar(
    y=roi_chart_data['LABEL'],
    x=roi_chart_data['ROI_MEAN'],
    orientation='h',
    marker_color=[COLORS['positive'] if sig else COLORS['neutral'] 
                  for sig in roi_chart_data['IS_SIGNIFICANT']],
    error_x=dict(
        type='data',
        symmetric=False,
        array=roi_chart_data[f'ROI_CI_UPPER_{ci_level}'] - roi_chart_data['ROI_MEAN'],
        arrayminus=roi_chart_data['ROI_MEAN'] - roi_chart_data[f'ROI_CI_LOWER_{ci_level}'],
        color='rgba(0,0,0,0.3)'
    ),
    hovertemplate='<b>%{y}</b><br>ROI: %{x:.2f}x<extra></extra>'
))

fig_roi.add_vline(x=1.0, line_dash="dash", line_color=COLORS['warning'],
                  annotation_text="Break-even (1.0x)", annotation_position="top right")

fig_roi.update_layout(
    title=dict(
        text=f"<b>Channel ROI Rankings</b><br><sup>Labels show total spend | Green = statistically significant</sup>",
        x=0.5, xanchor='center'
    ),
    xaxis_title="Return on Investment (ROI)",
    yaxis_title="Channel (Total Spend)",
    height=500,
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)
fig_roi.update_xaxes(gridcolor='lightgray', zeroline=True, zerolinecolor='gray')
st.plotly_chart(fig_roi, use_container_width=True)

# ─────────────────────────────────────────────────────────────────────────────
# PANEL 2: ROI vs Spend Scatter (The Key Insight)
# ─────────────────────────────────────────────────────────────────────────────
st.subheader("2. ROI vs Total Spend")
st.caption("📊 Notice: Low-spend channels (left side) often show extreme ROI values - this is the signal reliability issue")

scatter_data = roi_confidence.copy()
scatter_data['SPEND_MILLIONS'] = scatter_data['TOTAL_SPEND'] / 1_000_000

fig_scatter_roi = go.Figure()

fig_scatter_roi.add_trace(go.Scatter(
    x=scatter_data['SPEND_MILLIONS'],
    y=scatter_data['ROI_MEAN'],
    mode='markers+text',
    marker=dict(
        size=scatter_data['SPEND_MILLIONS'] / scatter_data['SPEND_MILLIONS'].max() * 40 + 10,
        color=[COLORS['positive'] if sig else COLORS['neutral'] for sig in scatter_data['IS_SIGNIFICANT']],
        line=dict(width=1, color='white')
    ),
    text=scatter_data['CHANNEL_KEY'].str.replace('_GLOBAL_ALL', ''),
    textposition='top center',
    textfont=dict(size=9),
    hovertemplate='<b>%{text}</b><br>Spend: $%{x:.1f}M<br>ROI: %{y:.2f}x<extra></extra>'
))

fig_scatter_roi.add_hline(y=1.0, line_dash="dash", line_color=COLORS['warning'],
                          annotation_text="Break-even (1.0x)")

# Add reliability zones
max_spend_val = scatter_data['SPEND_MILLIONS'].max()
fig_scatter_roi.add_vrect(x0=0, x1=max_spend_val*0.15, fillcolor="rgba(255,0,0,0.1)", 
                          line_width=0, annotation_text="⚠️ Low confidence",
                          annotation_position="top left")

fig_scatter_roi.update_layout(
    title=dict(
        text="<b>Spend Level vs ROI Estimate</b><br><sup>Red zone = low spend = unreliable ROI estimates</sup>",
        x=0.5, xanchor='center'
    ),
    xaxis_title="Total Spend ($M)",
    yaxis_title="ROI (Model Estimate)",
    height=500,
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)
fig_scatter_roi.update_xaxes(gridcolor='lightgray')
fig_scatter_roi.update_yaxes(gridcolor='lightgray')
st.plotly_chart(fig_scatter_roi, use_container_width=True)

# ─────────────────────────────────────────────────────────────────────────────
# PANEL 3: Filtered ROI View (Interactive)
# ─────────────────────────────────────────────────────────────────────────────
st.subheader("3. Filtered ROI View (Reliable Estimates Only)")

min_spend_val = roi_confidence['TOTAL_SPEND'].min() / 1e6
max_spend_val = roi_confidence['TOTAL_SPEND'].max() / 1e6
default_threshold = max(min_spend_val, max_spend_val * 0.10)

spend_threshold = st.slider(
    "Minimum spend threshold ($M)",
    min_value=float(min_spend_val),
    max_value=float(max_spend_val * 0.5),
    value=float(default_threshold),
    step=1.0,
    help="Filter out low-spend channels with unreliable ROI estimates"
)

filtered_data = roi_confidence[roi_confidence['TOTAL_SPEND'] >= spend_threshold * 1e6].copy()
filtered_data = filtered_data.sort_values('ROI_MEAN', ascending=True)

if len(filtered_data) > 0:
    st.info(f"Showing {len(filtered_data)} channels with spend ≥ ${spend_threshold:.0f}M")
    
    fig_filtered = go.Figure()
    
    fig_filtered.add_trace(go.Bar(
        y=filtered_data['CHANNEL_KEY'],
        x=filtered_data['ROI_MEAN'],
        orientation='h',
        marker_color=[COLORS['positive'] if sig else COLORS['neutral'] 
                      for sig in filtered_data['IS_SIGNIFICANT']],
        error_x=dict(
            type='data',
            symmetric=False,
            array=filtered_data[f'ROI_CI_UPPER_{ci_level}'] - filtered_data['ROI_MEAN'],
            arrayminus=filtered_data['ROI_MEAN'] - filtered_data[f'ROI_CI_LOWER_{ci_level}'],
            color='rgba(0,0,0,0.3)'
        ),
        hovertemplate='<b>%{y}</b><br>ROI: %{x:.2f}x<extra></extra>'
    ))
    
    fig_filtered.add_vline(x=1.0, line_dash="dash", line_color=COLORS['warning'])
    
    fig_filtered.update_layout(
        title=dict(
            text=f"<b>ROI Rankings (Spend ≥ ${spend_threshold:.0f}M)</b><br><sup>These estimates have sufficient signal for reliability</sup>",
            x=0.5, xanchor='center'
        ),
        xaxis_title="Return on Investment (ROI)",
        yaxis_title="Channel",
        height=400,
        showlegend=False,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )
    fig_filtered.update_xaxes(gridcolor='lightgray', zeroline=True, zerolinecolor='gray')
    st.plotly_chart(fig_filtered, use_container_width=True)
else:
    st.warning("No channels meet the spend threshold. Lower the threshold to see results.")

st.markdown("""
---
**Key Takeaway:** The model reliably identifies ROI for high-spend channels. 
For low-spend channels showing extreme ROI values, we recommend:
- **Geo-holdout tests** to validate performance
- **Incrementality testing** before scaling spend
- **Treating model estimates as directional** rather than precise
""")

In [ ]:
# =============================================================================
# CELL 13c: Additional Visualizations (Model Fit, Response Curves, etc.)
# =============================================================================
#
# This cell contains supplementary visualizations with INTERACTIVE FILTERS:
# - Model Fit (Actual vs Predicted)
# - Response Curves (Diminishing Returns) - all channels selectable
# - Channel Metrics Table (Technical Analysis)
# - Channel Contribution Treemap
# =============================================================================

import math

# ─────────────────────────────────────────────────────────────────────────────
# GLOBAL FILTERS FOR THIS SECTION
# ─────────────────────────────────────────────────────────────────────────────
st.markdown("---")
st.header("📊 Detailed Analysis")

# Prepare channel list sorted by spend
all_channels_sorted = sorted(channels, key=lambda ch: X_media[ch].sum(), reverse=True)
channel_display_names = {ch: ch.replace('_GLOBAL_ALL', '').replace('_', ' ') for ch in all_channels_sorted}

# Top N channels by spend (for default selection)
top_6_by_spend = all_channels_sorted[:6]

# Channel selection
st.subheader("Channel Selection")
col_filter1, col_filter2 = st.columns([3, 1])

with col_filter1:
    selected_channels = st.multiselect(
        "Select channels to analyze",
        options=all_channels_sorted,
        default=top_6_by_spend,
        format_func=lambda x: channel_display_names[x],
        help="Choose which channels to show in response curves and metrics table"
    )

with col_filter2:
    if st.button("Select All"):
        selected_channels = all_channels_sorted
    if st.button("Top 6 by Spend"):
        selected_channels = top_6_by_spend

if len(selected_channels) == 0:
    st.warning("Please select at least one channel to display.")
    selected_channels = top_6_by_spend  # Fallback

st.info(f"Analyzing {len(selected_channels)} channels")

# ─────────────────────────────────────────────────────────────────────────────
# CHART: Model Fit - Predicted vs Actual (Aggregated Weekly)
# ─────────────────────────────────────────────────────────────────────────────
st.subheader("4. Model Fit: Predicted vs Actual Revenue")

# Get predictions from the trained model (y and y_pred are at the same level)
X_media_trans = apply_media_transformations(X_media, best_params, channels)
X_full_viz = pd.concat([X_media_trans, X_control], axis=1)
X_scaled_viz = scaler.transform(X_full_viz)
y_pred_viz = model.predict(X_scaled_viz)

# y and y_pred are aligned - get corresponding weeks
# Note: y was created from aggregated weekly data, so len(y) == number of weeks
weeks = df.groupby('WEEK_START').first().reset_index()['WEEK_START'].values[:len(y)]

weekly_fit = pd.DataFrame({
    'WEEK_START': weeks,
    'Actual': y.values,
    'Predicted': y_pred_viz
})

# Calculate weekly metrics
correlation = weekly_fit['Actual'].corr(weekly_fit['Predicted'])
weekly_mape = np.mean(np.abs((weekly_fit['Actual'] - weekly_fit['Predicted']) / weekly_fit['Actual'])) * 100

fig_fit = go.Figure()

# Actual revenue
fig_fit.add_trace(go.Scatter(
    x=weekly_fit['WEEK_START'],
    y=weekly_fit['Actual'],
    mode='lines',
    name='Actual Revenue',
    line=dict(color=COLORS['primary'], width=2),
    fill='tozeroy',
    fillcolor='rgba(0, 104, 201, 0.1)'
))

# Predicted revenue
fig_fit.add_trace(go.Scatter(
    x=weekly_fit['WEEK_START'],
    y=weekly_fit['Predicted'],
    mode='lines',
    name='Predicted Revenue',
    line=dict(color=COLORS['accent'], width=2, dash='dash')
))

fig_fit.update_layout(
    title=dict(
        text=f"<b>Model Fit: Actual vs Predicted Weekly Revenue</b><br><sup>Correlation: {correlation:.2f} | Weekly MAPE: {weekly_mape:.1f}%</sup>",
        x=0.5, xanchor='center'
    ),
    xaxis_title="Week",
    yaxis_title="Revenue ($)",
    height=450,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    hovermode='x unified'
)
fig_fit.update_xaxes(gridcolor='lightgray')
fig_fit.update_yaxes(gridcolor='lightgray', tickformat='$,.0f')
st.plotly_chart(fig_fit, use_container_width=True)

# Interpretation help
if correlation > 0.7:
    st.success(f"✓ Good model fit - predicted revenue tracks actual revenue well (r={correlation:.2f})")
elif correlation > 0.4:
    st.warning(f"⚠ Moderate model fit - some patterns captured but significant unexplained variance (r={correlation:.2f})")
else:
    st.error(f"✗ Poor model fit - model not capturing revenue patterns (r={correlation:.2f}). Results should be interpreted with caution.")

# ─────────────────────────────────────────────────────────────────────────────
# CHART: Response Curves (Diminishing Returns) - WEEKLY SPEND BASIS
# ─────────────────────────────────────────────────────────────────────────────
st.subheader("5. Response Curves: Diminishing Returns")
st.markdown("""
**How to read:** X-axis is what you **spend per week** (input). Y-axis is the **revenue generated per week** (output).  
The curve shows how additional spend produces less additional revenue (diminishing returns).  
**Dashed orange line** = your current average weekly spend level.
""")

# Dynamic grid based on number of selected channels
n_channels = len(selected_channels)
n_cols = min(3, n_channels)  # Max 3 columns
n_rows = math.ceil(n_channels / n_cols) if n_cols > 0 else 1

fig_response = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=[channel_display_names[ch] for ch in selected_channels],
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

for idx, ch in enumerate(selected_channels):
    row = idx // n_cols + 1
    col = idx % n_cols + 1
    
    ch_curves = response_curves[response_curves['CHANNEL'] == ch].sort_values('SPEND')
    
    if len(ch_curves) > 0:
        # Plot the response curve
        fig_response.add_trace(
            go.Scatter(
                x=ch_curves['SPEND'],
                y=ch_curves['PREDICTED_REVENUE'],
                mode='lines',
                line=dict(color=COLORS['primary'], width=2),
                name=ch,
                showlegend=False,
                hovertemplate='Spend: $%{x:,.0f}/wk<br>Revenue: $%{y:,.0f}/wk<extra></extra>'
            ),
            row=row, col=col
        )
        
        # Current WEEKLY spend (mean, not sum) - matches how curves are generated
        weekly_spend = X_media[ch].mean()
        
        # Add vertical line at current weekly spend
        fig_response.add_vline(
            x=weekly_spend,
            line=dict(color=COLORS['accent'], width=2, dash='dash'),
            row=row, col=col
        )

# Dynamic height based on rows
chart_height = 300 * n_rows

fig_response.update_layout(
    title=dict(
        text="<b>Spend → Revenue Response Curves</b><br><sup>Curves flatten = diminishing returns | Orange line = current spend level</sup>",
        x=0.5, xanchor='center'
    ),
    height=chart_height,
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

# Update axes for all subplots
for i in range(1, n_channels + 1):
    row = (i - 1) // n_cols + 1
    col = (i - 1) % n_cols + 1
    fig_response.update_xaxes(tickformat='$,.0s', gridcolor='lightgray', title_text="$ Spend / Week →", row=row, col=col)
    fig_response.update_yaxes(tickformat='$,.0s', gridcolor='lightgray', title_text="← $ Revenue / Week", row=row, col=col)

st.plotly_chart(fig_response, use_container_width=True)

# ─────────────────────────────────────────────────────────────────────────────
# TABLE: Channel Response Curve Metrics (Technical Analysis)
# ─────────────────────────────────────────────────────────────────────────────
st.subheader("6. Channel Response Curve Analysis")
st.markdown("""
**Technical metrics from the fitted response curves.** These help identify where each channel sits 
on its diminishing returns curve and the key parameters driving the shape.
""")

# Build metrics table from response curves and model parameters
channel_metrics = []

for ch in selected_channels:  # Use the same channels shown in response curves
    p = best_params[ch]
    
    # Current weekly spend
    weekly_spend = X_media[ch].mean()
    
    # Get response curve data for this channel
    ch_curves = response_curves[response_curves['CHANNEL'] == ch].sort_values('SPEND')
    
    if len(ch_curves) > 0:
        # Revenue ceiling (max of predicted revenue)
        revenue_ceiling = ch_curves['PREDICTED_REVENUE'].max()
        
        # Current revenue (interpolate from curve at current spend)
        closest_idx = (ch_curves['SPEND'] - weekly_spend).abs().argmin()
        current_revenue = ch_curves.iloc[closest_idx]['PREDICTED_REVENUE']
        
        # Marginal ROI at current spend
        current_marginal_roi = ch_curves.iloc[closest_idx]['MARGINAL_ROI_AT_SPEND']
        
        # % of ceiling captured
        pct_of_ceiling = (current_revenue / revenue_ceiling * 100) if revenue_ceiling > 0 else 0
        
        # Half-saturation point (gamma) - spend level where you hit 50% of max response
        # This is where diminishing returns really accelerate
        half_sat_spend = p['gamma']
        
        # How far past half-saturation are we? (negative = room to grow, positive = saturated)
        saturation_ratio = weekly_spend / half_sat_spend if half_sat_spend > 0 else 0
        
        # Classify position
        if saturation_ratio < 0.5:
            position = "🟢 Early (high growth)"
        elif saturation_ratio < 1.0:
            position = "🟡 Approaching inflection"
        elif saturation_ratio < 2.0:
            position = "🟠 Past inflection"
        else:
            position = "🔴 Deep saturation"
        
        channel_metrics.append({
            'Channel': ch.replace('_GLOBAL_ALL', ''),
            'Spend/Wk': f"${weekly_spend:,.0f}",
            'Revenue Ceiling': f"${revenue_ceiling:,.0f}/wk",
            '% of Ceiling': f"{pct_of_ceiling:.0f}%",
            'Marginal ROI': f"{current_marginal_roi:.2f}x",
            'Half-Sat Point (γ)': f"${half_sat_spend:,.0f}",
            'vs Half-Sat': f"{saturation_ratio:.1f}x",
            'Decay Rate (θ)': f"{p['theta']:.2f}",
            'Position': position
        })

metrics_df = pd.DataFrame(channel_metrics)

# Add sorting options
sort_col1, sort_col2 = st.columns([2, 2])
with sort_col1:
    sort_by = st.selectbox(
        "Sort table by",
        options=['Spend/Wk', 'Marginal ROI', 'vs Half-Sat', '% of Ceiling', 'Position'],
        index=0
    )

# Create numeric columns for sorting
metrics_df['_spend_num'] = metrics_df['Spend/Wk'].str.replace(r'[$,]', '', regex=True).astype(float)
metrics_df['_marginal_num'] = metrics_df['Marginal ROI'].str.replace('x', '').astype(float)
metrics_df['_vs_halfsat_num'] = metrics_df['vs Half-Sat'].str.replace('x', '').astype(float)
metrics_df['_pct_ceiling_num'] = metrics_df['% of Ceiling'].str.replace('%', '').astype(float)

sort_mapping = {
    'Spend/Wk': ('_spend_num', False),
    'Marginal ROI': ('_marginal_num', False),
    'vs Half-Sat': ('_vs_halfsat_num', True),
    '% of Ceiling': ('_pct_ceiling_num', False),
    'Position': ('_vs_halfsat_num', True)
}
sort_field, ascending = sort_mapping[sort_by]
metrics_df = metrics_df.sort_values(sort_field, ascending=ascending)

# Drop helper columns
display_df = metrics_df.drop(columns=['_spend_num', '_marginal_num', '_vs_halfsat_num', '_pct_ceiling_num'])

st.dataframe(
    display_df,
    use_container_width=True,
    hide_index=True
)

# Explanation of key columns
with st.expander("📖 How to interpret these metrics"):
    st.markdown("""
    | Metric | What it means |
    |--------|---------------|
    | **Spend/Wk** | Your current average weekly spend on this channel |
    | **Revenue Ceiling** | Maximum weekly revenue this channel can deliver (curve asymptote) |
    | **% of Ceiling** | How much of the channel's potential you're currently capturing |
    | **Marginal ROI** | Return on the *next* dollar spent. <1.0 means losing money on incremental spend |
    | **Half-Sat Point (γ)** | The gamma parameter - spend level where you hit 50% of max response. This is where diminishing returns accelerate significantly |
    | **vs Half-Sat** | Your spend ÷ half-saturation point. <1.0 = room to grow, >1.0 = past the inflection point |
    | **Decay Rate (θ)** | The theta parameter - how much of this week's ad effect carries into next week. Higher = longer-lasting effect (good for awareness channels) |
    | **Position** | Quick assessment based on where you sit relative to the half-saturation point |
    
    **Key insight**: Channels where you're spending BELOW the half-saturation point (vs Half-Sat < 1.0) 
    have the most headroom for efficient growth. Channels well ABOVE it are in diminishing returns territory.
    """)

# ─────────────────────────────────────────────────────────────────────────────
# CHART: Channel Contribution Treemap
# ─────────────────────────────────────────────────────────────────────────────
st.subheader("7. Channel Contribution Breakdown")

treemap_col1, treemap_col2 = st.columns([2, 2])
with treemap_col1:
    treemap_scope = st.radio(
        "Show channels",
        options=['All Channels', 'Selected Channels Only'],
        horizontal=True
    )

contrib_data = model_results[['CHANNEL_CODE', 'CURRENT_SPEND', 'ROI']].copy()
contrib_data['CONTRIBUTION'] = contrib_data['CURRENT_SPEND'] * contrib_data['ROI']
contrib_data = contrib_data.groupby('CHANNEL_CODE').agg({
    'CURRENT_SPEND': 'sum',
    'CONTRIBUTION': 'sum',
    'ROI': 'mean'
}).reset_index()

# Filter to selected channels if requested
if treemap_scope == 'Selected Channels Only':
    # Map selected channels to channel codes (strip _GLOBAL_ALL suffix)
    selected_codes = [ch.replace('_GLOBAL_ALL', '') for ch in selected_channels]
    contrib_data = contrib_data[contrib_data['CHANNEL_CODE'].isin(selected_codes)]

contrib_data = contrib_data.sort_values('CONTRIBUTION', ascending=False)

fig_treemap = px.treemap(
    contrib_data,
    path=['CHANNEL_CODE'],
    values='CONTRIBUTION',
    color='ROI',
    color_continuous_scale='RdYlGn',
    color_continuous_midpoint=1.0,
    hover_data={'CURRENT_SPEND': ':$,.0f', 'ROI': ':.2f'}
)

fig_treemap.update_layout(
    title=dict(
        text="<b>Revenue Contribution by Channel</b><br><sup>Size = attributed revenue | Color = ROI (green = high, red = low)</sup>",
        x=0.5, xanchor='center'
    ),
    height=500,
    coloraxis_colorbar=dict(title="ROI")
)
st.plotly_chart(fig_treemap, use_container_width=True)

st.success("All visualizations complete!")

In [ ]:
# =============================================================================
# CELL 13: Save Results to Snowflake
# =============================================================================
#
# THREE OUTPUT TABLES (all overwrite mode for idempotent execution):
#
# 1. MMM.MODEL_RESULTS (overwrite mode)
#    - Channel-level results with ROI, confidence intervals, and parameters
#    - Used by: Streamlit app, analysis views
#
# 2. MMM.RESPONSE_CURVES (overwrite mode)
#    - Detailed spend → revenue curves for visualization
#    - 100 points per channel (0 to 3x max spend)
#    - Used by: Streamlit "What-If Simulator" charts
#
# 3. MMM.MODEL_METADATA (overwrite mode)
#    - Model configuration and quality metrics
#    - Tracks: R², MAPE, hyperparameter settings
#    - Used for: Model quality monitoring
#
# All tables use OVERWRITE to ensure clean, idempotent results on each run.
# Historical tracking can be done via a separate versioning/archival process.
# =============================================================================

def save_to_snowflake(session, model_results, response_curves, config, metrics):
    """
    Save model results and response curves to Snowflake.
    
    OUTPUT TABLES:
    - MMM.MODEL_RESULTS: Channel-level ROI with confidence intervals and parameters
    - MMM.RESPONSE_CURVES: Detailed curves with CI bands and efficiency zones
    - MMM.MODEL_METADATA: Model configuration and quality metrics
    """
    print("\nSaving results to Snowflake...")
    
    # 1. Save main results to MMM.MODEL_RESULTS (enhanced schema)
    results_clean = model_results.copy()
    
    # Map DataFrame columns to table columns
    results_for_db = pd.DataFrame({
        'MODEL_VERSION': results_clean['MODEL_VERSION'],
        'CHANNEL': results_clean['CHANNEL_KEY'],
        'COEFF_WEIGHT': results_clean['COEFFICIENT_WEIGHT'],
        'ROI': results_clean['ROI'],
        'MARGINAL_ROI': results_clean['MARGINAL_ROI'],
        'OPTIMAL_SPEND': results_clean['OPTIMAL_SPEND_SUGGESTION'],
        # Confidence intervals
        'ROI_CI_LOWER': results_clean['ROI_CI_LOWER'],
        'ROI_CI_UPPER': results_clean['ROI_CI_UPPER'],
        'IS_SIGNIFICANT': results_clean['IS_SIGNIFICANT'],
        # Learned parameters
        'ADSTOCK_DECAY': results_clean['ADSTOCK_DECAY_RATE'],
        'SATURATION_ALPHA': results_clean['SATURATION_ALPHA'],
        'SATURATION_GAMMA': results_clean['SATURATION_POINT'],
        # Model quality
        'CV_MAPE': results_clean['MODEL_MAPE_CV'],
        'R_SQUARED': results_clean['MODEL_R2_INSAMPLE'],
        'N_OBSERVATIONS': results_clean['N_OBSERVATIONS'],
        # Spend context
        'CURRENT_SPEND': results_clean['CURRENT_SPEND'],
        'SPEND_SHARE': results_clean['SPEND_SHARE']
    })
    
    results_sf = session.create_dataframe(results_for_db)
    results_sf.write.mode("overwrite").save_as_table("MMM.MODEL_RESULTS")
    print(f"  ✓ Saved {len(results_for_db)} rows to MMM.MODEL_RESULTS")
    
    # Legacy ATOMIC table save removed - schema incompatible with flat output
    # Use MMM.MODEL_RESULTS as the primary results table
    print(f"  ✓ Skipping legacy ATOMIC.MMM_MODEL_RESULT (use MMM.MODEL_RESULTS instead)")
    
    # 2. Save enhanced response curves (with CI bands and efficiency zones)
    curves_clean = response_curves.copy()
    curves_clean['MODEL_VERSION'] = config.model_version
    
    curves_sf = session.create_dataframe(curves_clean)
    curves_sf.write.mode("overwrite").save_as_table("MMM.RESPONSE_CURVES")
    print(f"  ✓ Saved {len(curves_clean)} rows to MMM.RESPONSE_CURVES")
    print(f"    → Includes: CI bands, marginal ROI at each point, efficiency zones")
    
    # 3. Save model metadata
    metadata = pd.DataFrame([{
        'MODEL_VERSION': config.model_version,
        'MODEL_RUN_DATE': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'GEO_LEVEL': config.geo_level,
        'PRODUCT_LEVEL': config.product_level,
        'N_CHANNELS': len(model_results),
        'R2_INSAMPLE': metrics['in_sample']['R2'],
        'MAPE_CV': metrics['cv_mean'].get('MAPE', None),
        'NEVERGRAD_BUDGET': config.nevergrad_budget,
        'N_BOOTSTRAP': config.n_bootstrap,
        'CONFIDENCE_LEVEL': config.confidence_level
    }])
    
    metadata_sf = session.create_dataframe(metadata)
    metadata_sf.write.mode("overwrite").save_as_table("MMM.MODEL_METADATA")
    print(f"  ✓ Saved model metadata to MMM.MODEL_METADATA")
    
    print("\n" + "="*60)
    print("SAVE COMPLETE")
    print("="*60)
    print(f"\nEnhanced outputs include:")
    print(f"  • 90% confidence intervals on all ROI estimates")
    print(f"  • Learned adstock decay and saturation parameters per channel")
    print(f"  • Response curve CI bands and efficiency zone classifications")

# Save to Snowflake
save_to_snowflake(session, model_results, response_curves, config, metrics)


In [ ]:
# =============================================================================
# CELL 14a: Save Transformed Features for SQL Inference
# =============================================================================
#
# PURPOSE: Persist the transformed features (adstock + saturation applied) 
# to enable SQL-based inference using MMM_CHANNEL_ROI model.
#
# WHY THIS IS NEEDED:
# The base model (MMM_CHANNEL_ROI) expects pre-transformed features. SQL cannot
# compute adstock transformations row-by-row because adstock requires temporal
# context (each value depends on all previous values).
#
# By saving the transformed features, we enable:
#   1. SQL inference: SELECT MMM_CHANNEL_ROI!PREDICT(...) FROM MMM_FEATURES_TRANSFORMED
#   2. Historical tracking of features used in training
#   3. Debugging/validation of transformation logic
#
# OUTPUT TABLE: MMM.MMM_FEATURES_TRANSFORMED
# =============================================================================

def save_transformed_features(session, X_transformed, X_control, y, df, channels, config):
    """
    Save the transformed features (adstock + saturation applied) to Snowflake.
    This enables SQL inference using the base MMM_CHANNEL_ROI model.
    """
    print("\n" + "="*60)
    print("SAVING TRANSFORMED FEATURES FOR SQL INFERENCE")
    print("="*60)
    
    # Get week dates
    weeks = df.groupby('WEEK_START').first().reset_index()['WEEK_START'].values[:len(y)]
    
    # Sanitize column names for SQL compatibility
    import re
    def sanitize_sql_identifier(name):
        sanitized = re.sub(r'[^A-Za-z0-9_]', '_', name)
        if sanitized and sanitized[0].isdigit():
            sanitized = '_' + sanitized
        return re.sub(r'_+', '_', sanitized)
    
    # Build the features dataframe
    features_df = pd.DataFrame({
        'MODEL_VERSION': config.model_version,
        'WEEK_START': weeks,
        'ACTUAL_REVENUE': y.values
    })
    
    # Add transformed media columns (these have adstock + saturation applied)
    for ch in channels:
        col_name = sanitize_sql_identifier(ch)
        features_df[col_name] = X_transformed[ch].values
    
    # Add control columns
    for col in X_control.columns:
        col_name = sanitize_sql_identifier(col)
        features_df[col_name] = X_control[col].values
    
    # Save to Snowflake
    features_sf = session.create_dataframe(features_df)
    features_sf.write.mode("overwrite").save_as_table("MMM.MMM_FEATURES_TRANSFORMED")
    
    print(f"  ✓ Saved {len(features_df)} rows to MMM.MMM_FEATURES_TRANSFORMED")
    print(f"  Columns: {len(features_df.columns)}")
    print(f"    - Model version + metadata: 3")
    print(f"    - Transformed media channels: {len(channels)}")
    print(f"    - Control variables: {len(X_control.columns)}")
    print(f"\n  SQL INFERENCE ENABLED:")
    print(f"    SELECT MMM.MMM_CHANNEL_ROI!PREDICT(...)")
    print(f"    FROM MMM.MMM_FEATURES_TRANSFORMED")
    print(f"    WHERE PMI_INDEX IS NOT NULL;")
    
    return features_df

# Save transformed features
features_saved = save_transformed_features(
    session, X_transformed, X_control, y, df, channels, config
)

In [ ]:
# =============================================================================
# CELL 14a-ALT: Save Features to Snowflake Feature Store (Demo)
# =============================================================================
#
# Alternative approach using Feature Store for:
#   - Built-in versioning and lineage
#   - Point-in-time feature retrieval
#   - Integration with Model Registry
#
# =============================================================================

from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

def save_to_feature_store(session, X_transformed, y, df, config, scaler):
    print('\n' + '='*60)
    print('SAVING TO SNOWFLAKE FEATURE STORE')
    print('='*60)
    
    # Initialize Feature Store (create schema if needed)
    fs = FeatureStore(
        session=session,
        database='GLOBAL_B2B_MMM',
        name='MMM_FEATURE_STORE',
        default_warehouse='GLOBAL_B2B_MMM_WH',
        creation_mode=CreationMode.CREATE_IF_NOT_EXIST
    )
    print('  Feature Store initialized')
    
    # Create Entity (the join key for features)
    week_entity = Entity(name='WEEK', join_keys=['WEEK_START'])
    fs.register_entity(week_entity)
    print('  Entity WEEK registered')
    
    # Prepare feature data
    weeks = df.groupby('WEEK_START').first().reset_index()['WEEK_START'].values[:len(y)]
    X_scaled = scaler.transform(X_transformed.values)
    
    import re
    def sanitize(name):
        s = re.sub(r'[^A-Za-z0-9_]', '_', name)
        return re.sub(r'_+', '_', s).upper()  # UPPERCASE for Snowflake
    
    features_df = pd.DataFrame({'WEEK_START': weeks})
    for idx, col in enumerate(X_transformed.columns):
        features_df[sanitize(col)] = X_scaled[:, idx]
    
    # Create source table for Feature View
    version_safe = config.model_version.replace('.', '_')
    source_table = f'MMM.MMM_FEATURES_SOURCE_{version_safe}'
    session.create_dataframe(features_df).write.mode('overwrite').save_as_table(source_table)
    print(f'  Source table created: {source_table}')
    
    # Create Feature View
    fv = FeatureView(
        name='MMM_SCALED_FEATURES',
        entities=[week_entity],
        feature_df=session.table(source_table),
        desc=f'Scaled MMM features for model {config.model_version}'
    )
    
    # Register with versioning
    registered_fv = fs.register_feature_view(
        feature_view=fv,
        version=version_safe,
        block=True
    )
    print(f'  Feature View registered: {registered_fv.name} v{registered_fv.version}')
    
    # Demo: retrieve features
    print('\n  DEMO RETRIEVAL:')
    spine_df = session.create_dataframe(pd.DataFrame({'WEEK_START': weeks[:5]}))
    retrieved = fs.retrieve_feature_values(
        spine_df=spine_df,
        features=[registered_fv]
    )
    print(f'  Retrieved {retrieved.count()} rows from Feature Store')
    
    return fs, registered_fv

# Run Feature Store demo
fs, fv = save_to_feature_store(session, X_transformed, y, df, config, scaler)

In [ ]:
# =============================================================================
# CELL 14b: Register Model to Snowflake Model Registry
# =============================================================================
#
# SNOWFLAKE MODEL REGISTRY
# ─────────────────────────────────────────────────────────────────────────────
# The Model Registry provides centralized model management:
#
# 1. VERSION TRACKING: Each training run creates a new version
# 2. METADATA STORAGE: Hyperparameters, metrics, and lineage
# 3. MODEL COMPARISON: Compare performance across versions in Snowsight
# 4. DEPLOYMENT: Models can be deployed for inference (optional)
#
# WHAT WE REGISTER:
# - The trained sklearn Ridge model
# - Quality metrics (R², CV MAPE)
# - Training configuration
#
# NOTE: sklearn models require `sample_input_data` to infer the model signature.
# We pass a small sample of the scaled feature matrix used during training.
#
# VIEW IN SNOWSIGHT: AI & ML → Models → mmm_channel_roi
# =============================================================================

def register_model_to_registry(session, model, scaler, X_transformed, config, metrics):
    """
    Register the trained MMM model to Snowflake Model Registry.
    
    This enables version tracking and model comparison in Snowsight UI.
    FAILS HARD if registration doesn't work - this is a critical step.
    
    Args:
        session: Snowflake session
        model: Trained sklearn Ridge model
        scaler: StandardScaler used to normalize features
        X_transformed: DataFrame with transformed features (before scaling)
        config: MMMConfig object
        metrics: Dictionary with model metrics
    
    Raises:
        Exception: If model registration fails for any reason
    """
    print("\n" + "="*60)
    print("SNOWFLAKE MODEL REGISTRY")
    print("="*60)
    
    from snowflake.ml.registry import Registry
    print("  ✓ Registry imported successfully")
    
    # Get current database for explicit registry location
    db_name = session.get_current_database()
    schema_name = "MMM"
    print(f"  Target location: {db_name}.{schema_name}")
    
    # Initialize registry with explicit database/schema
    print("  Initializing Registry...")
    reg = Registry(session=session, database_name=db_name, schema_name=schema_name)
    print("  ✓ Registry initialized")
    
    # Prepare metrics for registry
    registry_metrics = {
        "r2_insample": float(metrics['in_sample']['R2']),
        "mape_cv": float(metrics['cv_mean'].get('MAPE', 0)),
        "rmse_insample": float(metrics['in_sample'].get('RMSE', 0)),
        "nrmse_cv": float(metrics['cv_mean'].get('NRMSE', 0)),
    }
    
    # Clean version name (replace dots and underscores that might cause issues)
    version_name = config.model_version.replace(".", "_").replace(" ", "_")
    print(f"  Version name: {version_name}")
    print(f"  Model type: {type(model).__name__}")
    
    # Create sample input data for model signature inference
    # sklearn models require sample_input_data to register with Snowflake ML
    # Use first 10 rows of SCALED features (what the model actually sees)
    # IMPORTANT: Sanitize column names to be valid SQL identifiers
    # Valid SQL identifiers: start with letter/underscore, contain only letters/digits/underscores
    import re
    def sanitize_sql_identifier(name):
        # Replace any non-alphanumeric character (except underscore) with underscore
        sanitized = re.sub(r'[^A-Za-z0-9_]', '_', name)
        # Ensure it starts with a letter or underscore (not a digit)
        if sanitized and sanitized[0].isdigit():
            sanitized = '_' + sanitized
        # Collapse multiple underscores
        sanitized = re.sub(r'_+', '_', sanitized)
        return sanitized
    
    sanitized_columns = [sanitize_sql_identifier(col) for col in X_transformed.columns]
    
    X_scaled_sample = pd.DataFrame(
        scaler.transform(X_transformed.head(10)),
        columns=sanitized_columns
    )
    print(f"  Sample input shape: {X_scaled_sample.shape}")
    print(f"  Sample columns (first 5): {sanitized_columns[:5]}")
    
    # Log the model to registry
    print("  Calling reg.log_model()...")
    model_ref = reg.log_model(
        model=model,
        model_name="MMM_CHANNEL_ROI",
        version_name=version_name,
        conda_dependencies=["scikit-learn", "pandas", "numpy"],
        target_platforms=["WAREHOUSE"],
        metrics=registry_metrics,
        sample_input_data=X_scaled_sample,
        comment=f"MMM model trained on {config.input_view} with geo_level={config.geo_level}"
    )
    print(f"  ✓ log_model returned: {model_ref}")
    
    print(f"\n✓ Model registered to Snowflake Model Registry")
    print(f"  Database: {db_name}")
    print(f"  Schema: {schema_name}")
    print(f"  Model Name: MMM_CHANNEL_ROI")
    print(f"  Version: {version_name}")
    print(f"  Metrics: R²={registry_metrics['r2_insample']:.4f}, CV MAPE={registry_metrics['mape_cv']:.1f}%")
    print(f"\n  View in Snowsight: AI & ML → Models → {db_name}.{schema_name}.MMM_CHANNEL_ROI")
    
    return model_ref

# Register model to Snowflake Model Registry
# Pass scaler and X_transformed so we can provide sample_input_data for signature inference
model_ref = register_model_to_registry(session, model, scaler, X_transformed, config, metrics)

In [ ]:
# =============================================================================
# CELL 14d: ML Observability - Model Monitoring Setup
# =============================================================================
#
# SNOWFLAKE ML OBSERVABILITY
# ─────────────────────────────────────────────────────────────────────────────
# ML Observability tracks model behavior over time:
#
# 1. DRIFT DETECTION: Monitors if input features shift from training baseline
# 2. PERFORMANCE TRACKING: Tracks prediction accuracy as ground truth arrives
# 3. SEGMENT ANALYSIS: Monitor performance by region, channel, etc.
#
# WHAT WE SET UP:
# - Predictions logging table (MMM_MODEL_PREDICTIONS)
# - Model Monitor attached to the registered model
# - Segment monitoring by SUPER_REGION
#
# VIEW IN SNOWSIGHT: AI & ML → Models → MMM_CHANNEL_ROI → Monitors
# 
# Docs: https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/model-observability
# =============================================================================

def setup_ml_observability(session, config, X_transformed, y, y_pred, df_input):
    """
    Set up ML Observability for the MMM model.
    
    Creates:
    1. Predictions logging table with features, predictions, and actuals
    2. Baseline table for drift comparison
    3. Model Monitor for ongoing tracking
    
    Args:
        session: Snowflake session
        config: MMMConfig object
        X_transformed: Transformed features DataFrame
        y: Actual revenue values
        y_pred: Model predictions
        df_input: Original input DataFrame (for segment columns)
    """
    print("\n" + "="*60)
    print("ML OBSERVABILITY SETUP")
    print("="*60)
    
    db_name = session.get_current_database()
    schema_name = "MMM"
    model_name = "MMM_CHANNEL_ROI"
    version_name = config.model_version.replace(".", "_").replace(" ", "_")
    monitor_name = f"MMM_MONITOR_{version_name}".upper()
    
    # -------------------------------------------------------------------------
    # Step 1: Create Predictions Logging Table
    # -------------------------------------------------------------------------
    print("\n[1/4] Creating predictions logging table...")
    
    # Prepare prediction data with required columns
    pred_df = pd.DataFrame({
        'PREDICTION_ID': [f"pred_{i}_{datetime.now().strftime('%Y%m%d%H%M%S')}" for i in range(len(y))],
        'PREDICTION_TS': pd.Timestamp.now(),
        'PREDICTION': y_pred.flatten() if hasattr(y_pred, 'flatten') else y_pred,
        'ACTUAL': y.values if hasattr(y, 'values') else y,
    })
    
    # Add segment column (SUPER_REGION) if available
    if 'SUPER_REGION' in df_input.columns:
        pred_df['SUPER_REGION'] = df_input['SUPER_REGION'].values[:len(pred_df)]
        segment_col = 'SUPER_REGION'
    else:
        segment_col = None
    
    # Add key features for drift monitoring (top 5 media channels by spend)
    feature_cols = []
    for i, col in enumerate(X_transformed.columns[:10]):  # Top 10 features
        safe_col = f"FEATURE_{i}"
        pred_df[safe_col] = X_transformed[col].values[:len(pred_df)]
        feature_cols.append(safe_col)
    
    print(f"  Prediction records: {len(pred_df)}")
    print(f"  Feature columns: {len(feature_cols)}")
    print(f"  Segment column: {segment_col or 'None'}")
    
    # Convert to Snowpark DataFrame and write
    sp_pred_df = session.create_dataframe(pred_df)
    
    predictions_table = f"{db_name}.{schema_name}.MMM_MODEL_PREDICTIONS"
    sp_pred_df.write.mode("append").save_as_table(predictions_table)
    print(f"  ✓ Appended to: {predictions_table}")
    
    # Ensure PREDICTION_TS is TIMESTAMP_NTZ (required by Model Monitor)
    session.sql(f"""
        ALTER TABLE {predictions_table} 
        ALTER COLUMN PREDICTION_TS SET DATA TYPE TIMESTAMP_NTZ
    """).collect()
    
    # -------------------------------------------------------------------------
    # Step 2: Create Baseline Table (for drift comparison)
    # -------------------------------------------------------------------------
    print("\n[2/4] Creating baseline table for drift detection...")
    
    baseline_table = f"{db_name}.{schema_name}.MMM_MODEL_BASELINE"
    
    # Use first 70% of data as baseline (training period)
    baseline_size = int(len(pred_df) * 0.7)
    baseline_df = pred_df.head(baseline_size).copy()
    
    # Baseline only needs to be created once - don't overwrite if exists
    baseline_exists = len(session.sql(f"SHOW TABLES LIKE 'MMM_MODEL_BASELINE' IN SCHEMA {db_name}.{schema_name}").collect()) > 0
    
    if not baseline_exists:
        sp_baseline_df = session.create_dataframe(baseline_df)
        sp_baseline_df.write.mode("overwrite").save_as_table(baseline_table)
        print(f"  ✓ Created: {baseline_table} ({baseline_size} rows)")
        
        session.sql(f"""
            ALTER TABLE {baseline_table} 
            ALTER COLUMN PREDICTION_TS SET DATA TYPE TIMESTAMP_NTZ
        """).collect()
    else:
        print(f"  ✓ Baseline table already exists: {baseline_table} (preserved)")
    
    # -------------------------------------------------------------------------
    # Step 3: Create Model Monitor (one per model, not per version)
    # -------------------------------------------------------------------------
    print("\n[3/4] Creating Model Monitor...")
    
    # Use a single monitor name (not version-specific) so we don't create many monitors
    monitor_name = "MMM_MONITOR"
    
    # Check if monitor already exists
    existing_monitors = session.sql(f"SHOW MODEL MONITORS LIKE '{monitor_name}' IN SCHEMA {db_name}.{schema_name}").collect()
    
    if len(existing_monitors) > 0:
        print(f"  ✓ Monitor already exists: {monitor_name} (preserved)")
    else:
        # Create the Model Monitor
        # Syntax: https://docs.snowflake.com/en/sql-reference/sql/create-model-monitor
        segment_clause = f"SEGMENT_COLUMNS = ('{segment_col}')" if segment_col else ""
        
        create_monitor_sql = f"""
        CREATE MODEL MONITOR {schema_name}.{monitor_name} WITH
            MODEL = {model_name}
            VERSION = '{version_name}'
            FUNCTION = 'predict'
            SOURCE = {predictions_table}
            WAREHOUSE = {session.get_current_warehouse()}
            REFRESH_INTERVAL = '1 hour'
            AGGREGATION_WINDOW = '1 day'
            TIMESTAMP_COLUMN = PREDICTION_TS
            ID_COLUMNS = ('PREDICTION_ID')
            BASELINE = {baseline_table}
            PREDICTION_SCORE_COLUMNS = ('PREDICTION')
            ACTUAL_SCORE_COLUMNS = ('ACTUAL')
            {segment_clause}
        """
        
        try:
            session.sql(create_monitor_sql).collect()
            print(f"  ✓ Created Model Monitor: {monitor_name}")
        except Exception as e:
            print(f"  ⚠ Monitor creation failed: {str(e)}")
            print("    (This may require additional privileges or feature enablement)")
            return None
    
    # -------------------------------------------------------------------------
    # Step 4: Verify and Display Info
    # -------------------------------------------------------------------------
    print("\n[4/4] Verifying Model Monitor...")
    
    try:
        monitor_info = session.sql(f"DESCRIBE MODEL MONITOR {db_name}.{schema_name}.{monitor_name}").collect()
        print(f"  ✓ Monitor is active")
        
        # Show monitor status
        for row in monitor_info:
            if hasattr(row, 'property') and hasattr(row, 'value'):
                print(f"    {row['property']}: {row['value']}")
    except Exception as e:
        print(f"  Monitor verification: {str(e)[:80]}")
    
    print(f"""
✓ ML OBSERVABILITY CONFIGURED
─────────────────────────────────────────────────────────────────────────────
  Model:          {model_name} (version: {version_name})
  Monitor:        {monitor_name}
  Predictions:    {predictions_table}
  Baseline:       {baseline_table}
  Segments:       {segment_col or 'None'}
  
  METRICS TRACKED:
  • Performance:  RMSE, MAE, R² (comparing predictions to actuals)
  • Drift:        PSI, JS Divergence (feature distribution shifts)
  • Volume:       Prediction counts, null rates
  
  VIEW IN SNOWSIGHT:
  AI & ML → Models → {model_name} → Monitors → {monitor_name}
  
  QUERY METRICS (SQL):
  SELECT * FROM TABLE(MODEL_MONITOR_PERFORMANCE_METRIC(
      '{monitor_name}', 'RMSE', 'DAY',
      DATEADD('day', -30, CURRENT_TIMESTAMP()),
      CURRENT_TIMESTAMP()
  ));
─────────────────────────────────────────────────────────────────────────────
""")
    
    return monitor_name


# Set up ML Observability
# Only run if model was successfully registered
if 'model_ref' in dir() and model_ref is not None:
    # Get predictions for logging
    y_pred_final = model.predict(scaler.transform(X_transformed))
    
    monitor_name = setup_ml_observability(
        session=session,
        config=config,
        X_transformed=X_transformed,
        y=y,
        y_pred=y_pred_final,
        df_input=df
    )
    
    # -------------------------------------------------------------------------
    # Set this version as the default (uncomment to enable)
    # -------------------------------------------------------------------------
    version_name = config.model_version.replace(".", "_").replace(" ", "_").upper()
    session.sql(f"ALTER MODEL MMM.MMM_CHANNEL_ROI SET DEFAULT_VERSION = '{version_name}'").collect()
    print(f"✓ Set {version_name} as default model version")
else:
    print("⚠ Skipping ML Observability setup - model not registered")
    monitor_name = None

In [ ]:
# =============================================================================
# CELL 14: Executive Summary
# =============================================================================

print("\n" + "="*70)
print("                    MMM TRAINING EXECUTIVE SUMMARY")
print("="*70)

print(f"""
MODEL CONFIGURATION
-------------------
Version:           {config.model_version}
Geographic Level:  {config.geo_level}
Product Level:     {config.product_level}
Channels Modeled:  {len(channels)}

MODEL QUALITY
-------------
In-Sample R²:      {metrics['in_sample']['R2']:.4f}
CV MAPE:           {metrics['cv_mean'].get('MAPE', 0):.1f}% ± {metrics['cv_std'].get('MAPE', 0):.1f}%
CV R²:             {metrics['cv_mean'].get('R2', 0):.4f} ± {metrics['cv_std'].get('R2', 0):.4f}

TOP PERFORMING CHANNELS (by ROI)
--------------------------------""")

ci_level = int(config.confidence_level * 100)
top_channels = roi_confidence.head(5)
for _, row in top_channels.iterrows():
    sig = "*" if row['IS_SIGNIFICANT'] else ""
    print(f"  {row['CHANNEL_KEY']}: ROI = {row['ROI_MEAN']:.2f} [{row[f'ROI_CI_LOWER_{ci_level}']:.2f}, {row[f'ROI_CI_UPPER_{ci_level}']:.2f}] {sig}")

print(f"""
BUDGET OPTIMIZATION HIGHLIGHTS
------------------------------
Constraint: ±{config.budget_change_limit*100:.0f}% per channel (budget neutral)""")

if len(budget_recommendations) > 0:
    top_increase = budget_recommendations.head(3)
    top_decrease = budget_recommendations.tail(3)
    
    print("\nIncrease Spend:")
    for _, row in top_increase.iterrows():
        print(f"  {row['CHANNEL_KEY']}: +{row['CHANGE_PCT']:.1f}% (${row['CHANGE_AMOUNT']:,.0f})")
    
    print("\nDecrease Spend:")
    for _, row in top_decrease.iterrows():
        print(f"  {row['CHANNEL_KEY']}: {row['CHANGE_PCT']:.1f}% (${row['CHANGE_AMOUNT']:,.0f})")

monitor_status = f"Active ({monitor_name})" if monitor_name else "Not configured"

print(f"""
OUTPUT TABLES & REGISTRY
------------------------
• MMM.MODEL_RESULTS          - Channel ROI with confidence intervals
• MMM.RESPONSE_CURVES        - Spend vs. revenue curves  
• MMM.MODEL_METADATA         - Model configuration and quality
• MMM.MMM_MODEL_PREDICTIONS  - Predictions for ML Observability
• Model Registry             - MMM_CHANNEL_ROI (Snowsight: AI & ML → Models)
• Model Monitor              - {monitor_status}

ML OBSERVABILITY
----------------
• Drift Detection:           Monitors feature distribution shifts
• Performance Tracking:      RMSE, MAE, R² over time
• Segment Analysis:          By SUPER_REGION

NEXT STEPS
----------
1. Review results in DIMENSIONAL.V_MMM_RESULTS_ANALYSIS view
2. Visualize response curves in Streamlit app
3. Use Budget Optimizer for scenario planning
4. Compare model versions in Model Registry
5. Monitor model health in Snowsight (AI & ML → Models → Monitors)
6. Run 02_snowflake_ml_features.ipynb for FORECAST baseline comparison
""")

print("="*70)
print(f"Model training completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)


In [ ]:
# =============================================================================
# CELL 15: Fix Ownership (if run as ACCOUNTADMIN)
# =============================================================================
# If you ran this notebook as ACCOUNTADMIN, the tables won't be accessible
# to the project role. This cell fixes ownership so the Streamlit app works.

current_role = session.sql("SELECT CURRENT_ROLE()").collect()[0][0]

if current_role in ("ACCOUNTADMIN", "SYSADMIN"):
    print(f"Detected {current_role} - transferring ownership to project role...")
    
    project_role = "GLOBAL_B2B_MMM_ROLE"
    database = "GLOBAL_B2B_MMM"
    
    ownership_grants = [
        f"GRANT OWNERSHIP ON TABLE {database}.MMM.MODEL_RESULTS TO ROLE {project_role} COPY CURRENT GRANTS",
        f"GRANT OWNERSHIP ON TABLE {database}.MMM.RESPONSE_CURVES TO ROLE {project_role} COPY CURRENT GRANTS",
        f"GRANT OWNERSHIP ON TABLE {database}.MMM.MODEL_METADATA TO ROLE {project_role} COPY CURRENT GRANTS",
        f"GRANT OWNERSHIP ON TABLE {database}.MMM.MMM_MODEL_PREDICTIONS TO ROLE {project_role} COPY CURRENT GRANTS",
    ]
    
    for grant in ownership_grants:
        try:
            session.sql(grant).collect()
            table_name = grant.split("ON TABLE ")[1].split(" TO")[0]
            print(f"  ✓ Transferred: {table_name}")
        except Exception as e:
            print(f"  ⚠ Skipped (may not exist): {e}")
    
    print(f"\nOwnership transferred to {project_role}. Streamlit app should now work.")
else:
    print(f"Running as {current_role} - no ownership fix needed.")